In [5]:
import open3d as o3d
import matplotlib.pyplot as plt
import numpy as np
import json
import glob
from PIL import Image
import torchvision
import torch
from torchvision import transforms as T 
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import time
from numba import jit, prange
import copy

import argparse

# Load MaskRCNN 

def load_maskrcnn_model(model_path):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, 2)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    return model

def colored_ICP(source, target):
    
    voxel_radius = [0.04, 0.02, 0.01]
    max_iter = [50, 30, 14]
    current_transformation = np.identity(4)
    #print("3. Colored point cloud registration")
    for scale in range(3):
        iters = max_iter[scale]
        radius = voxel_radius[scale]
        #print("iteration: ", iters, radius, scale)

        #print("3-1. Downsample with a voxel size %.2f" % radius)
        source_down = source.voxel_down_sample(radius)
        target_down = target.voxel_down_sample(radius)

        #print("3-2. Estimate normal.")
        source_down.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
        target_down.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))

        #print("3-3. Applying colored point cloud registration")
        result_icp = o3d.pipelines.registration.registration_colored_icp(
            source_down, target_down, radius, current_transformation,
            o3d.pipelines.registration.TransformationEstimationForColoredICP(),
            o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                              relative_rmse=1e-6,
                                                              max_iteration=iters))
        current_transformation = result_icp.transformation
        return current_transformation
    

def get_model_mask(model_generic, image):
    proba_threshold = 0.5
    ig = transform(image)
    with torch.no_grad():
        prediction = model_generic([ig.to(device)])
        
    if(prediction[0]['masks'].nelement() == 0):
        XX = torch.empty((0,0), dtype=torch.int64)
        return XX
    predicted_mask = prediction[0]
    predicted_mask = predicted_mask['masks'][0] > proba_threshold
    
    predicted_mask = predicted_mask.squeeze(1)
    mask = predicted_mask.cpu().detach()
    return mask

def segment_images(last_frame,fctr,path):
    depth_PIL = Image.fromarray(np.asarray(last_frame.depth)).convert("RGB")
    rgb_image_array = np.asarray(last_frame.color)
    depth_image_array = np.asarray(last_frame.depth)
    rgb_PIL = Image.fromarray(rgb_image_array)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    rgb_mask = get_model_mask(model_rgb, rgb_PIL)
    depth_mask = get_model_mask(model_depth, depth_PIL)

    if depth_mask.nelement() == 0:
        mask_combined = rgb_mask
    else:
        mask_combined = depth_mask | rgb_mask  # 1 vote arbitration (OR the masks)

    # Convert tensor to numpy array and ensure the right datatype
    mask_combined = mask_combined.numpy().astype(rgb_image_array.dtype)
    mask_image = mask_combined.swapaxes(0, 2).swapaxes(0, 1)
    mask_image = (mask_image > 0).astype(rgb_image_array.dtype)   

    fg_image_rgb = rgb_image_array * mask_image

    # For the depth image:
    squeezed_mask = np.squeeze(mask_image)
    fg_image_depth = depth_image_array * squeezed_mask
    
    last_frame.color = o3d.geometry.Image(fg_image_rgb)
    last_frame.depth = o3d.geometry.Image(fg_image_depth)
     # Take this out!!
    o3d.io.write_image(path+"color_"+str(fctr)+".jpg",last_frame.color)
    o3d.io.write_image(path+"depth_"+str(fctr)+".png",last_frame.depth)

    return last_frame



def eliminate_flying_pixels(depth_image, ws, threshold):
   
    # Get image size
    height, width = depth_image.shape
    # Create an empty array for the result
    result = np.zeros_like(depth_image, dtype=float)

    # Iterate over the entire image
    for cy in range(height):
        for cx in range(width):
            # Set the range for the window
            x_start, x_end = max(0, cx - ws), min(width, cx + ws + 1)
            y_start, y_end = max(0, cy - ws), min(height, cy + ws + 1)
            
            # Get the window
            window = depth_image[y_start:y_end, x_start:x_end]

            # Calculate the sum of absolute differences
            result[cy, cx] = np.sum(np.abs(window - depth_image[cy, cx]))
    count = np.sum(result > threshold)

    depth_image[result > threshold] = 0
    return  depth_image

def filter_file_names(file_list): # to avoid the two head cameras
    new_file_list = []
    for f in file_list:
        tmp = f.split('.mkv')[0]
        ID  = tmp.split('_')[-1]
        if (ID == "13" or ID == "15"):
            continue
        else :
            new_file_list.append(f)
    return new_file_list




def load_filter_pcds(data_path,Tforms):  # returns a list of  pcds
    ws = 2
    flying_pixel_filter_threshold = 100

    reader = o3d.io.AzureKinectMKVReader()
    abspath = data_path
    pcds = []    
        
    files = glob.glob(data_path+'/*.mkv')
    #files = filter_file_names(files_raw)
    files.sort()
    list_size = len(files)
    rgbd_frames = [None] * list_size
  
    for i in range(len(files)): # for each view
   
        inFile = files[i]
        fname = inFile.split('/')[-1]
        file_name = fname.split('.mkv')[0]
        reader.open(inFile)
        if not reader.is_opened():
            raise RuntimeError("Unable to open file {}".format(inFile))
        metadata = reader.get_metadata()
  
        # write the metadata to a JSON file since that seems to be the only
        # way to retrieve that data
        o3d.io.write_azure_kinect_mkv_metadata(
                    '{}/{}_intrinsic.json'.format(abspath,file_name), metadata)

        # Open the file and load the JSON
        with open(abspath+"/" + file_name + "_intrinsic.json") as f:
            data = json.load(f)
        
        height = data['height']
        width = data['width']
        intrinsics = data["intrinsic_matrix"]
        #time_stamp = data["stream_length_usec"]
        #print(f"Intrinsic Matrix {intrinsics}")

        camera_intrinsics = o3d.camera.PinholeCameraIntrinsic()
        cx = intrinsics[6]
        cy = intrinsics[7]
        fx = intrinsics[0]
        fy = intrinsics[4]
        camera_intrinsics.set_intrinsics(width,height,fx,fy,cx,cy)
            
        last_frame = None
        while not reader.is_eof(): # go until hitting eof because of exposure issues in early color frames
            rgbda = reader.next_frame()
            if rgbda is None:
                #print("Got nothing! ")
                continue
            last_frame = rgbda

        if last_frame is not None:
            #print("Got the last frame")
            rgbd_frames[i] = last_frame
        else:
            
            print("************No valid frames found in the .mkv file.**********")
    
        # filter the depth image for flying pixels
        
        depth_image_array = np.asarray(last_frame.depth)
        result_mask = numba_eliminate_flying_pixels(depth_image_array.copy(), ws, flying_pixel_filter_threshold)
        depth_image_array[result_mask > flying_pixel_filter_threshold] = 0
        # Re-insert filtered depth into the rgbd image
        last_frame.depth = o3d.geometry.Image(depth_image_array)
      
        # Apply maskrcnn to filtered depth image
        masked_rgbd = segment_images(last_frame,i,data_path)
        rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        masked_rgbd.color, masked_rgbd.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
        pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbdc,
                                                            camera_intrinsics) 
        
        pcds.append(copy.deepcopy(pcd).transform(Tforms[i]))
        reader.close()
        
    return pcds

def backproject_o3d(rgbd_frame, intrinsics):
    
    rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        rgbd_frame.color, rgbd_frame.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)


    # Create a point cloud from the RGBD image
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbdc, intrinsics)
    n_radius = 0.01*2.0
    pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=n_radius, max_nn=30))
    # Visualize the point cloud
    #o3d.visualization.draw_geometries([pcd])
    
    return pcd


@jit(nopython=True, parallel=True)
def numba_eliminate_flying_pixels(depth_image, ws, threshold):
    height, width = depth_image.shape
    result = np.zeros_like(depth_image, dtype=np.float64)
    
    for cy in prange(height):
        for cx in prange(width):
            x_start, x_end = max(0, cx - ws), min(width, cx + ws + 1)
            y_start, y_end = max(0, cy - ws), min(height, cy + ws + 1)
            window = depth_image[y_start:y_end, x_start:x_end]
            result[cy, cx] = np.sum(np.abs(window - depth_image[cy, cx]))
    
    #for i in prange(height):
       # for j in prange(width):
      #      if result[i, j] > threshold:
     #           depth_image[i, j] = 0
    #count = np.sum(result > threshold)
    #print("Numba detected: #", count)
    return result



# Main
rgb_model_path = '/home/vigir3d/Datasets/cattle_scans/maskrcnn_data/maskrcnn_v2.pth'
depth_model_path = '/home/vigir3d/Datasets/cattle_scans/maskrcnn_data/maskrcnn_depth_best.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loads the maskrcnn trained for depth and rgb
model_rgb = load_maskrcnn_model(rgb_model_path)
model_depth = load_maskrcnn_model(depth_model_path)

transform = T.ToTensor()
def load_transformations(tform_path):
    """Load transformations from the given path."""
    htm_files = ["htm_0_3.txt", "htm_1_3.txt", "htm_5_3.txt", "htm_6_3.txt", "htm_7_3.txt"]
    transformations = [np.loadtxt(tform_path + file) for file in htm_files]
    transformations.insert(2, np.eye(4))
    return transformations

def apply_transformations(pcds_all):
    """Compute and apply transformations."""
    t01 = colored_ICP(pcds_all[0], pcds_all[1])
    t52 = colored_ICP(pcds_all[5], pcds_all[2])
    t43 = colored_ICP(pcds_all[4], pcds_all[3])
    t12 = colored_ICP(pcds_all[1], pcds_all[2])
    t32 = colored_ICP(pcds_all[3], pcds_all[2])

    H0 = t12 @ t01
    H1 = t12
    H3 = t32
    H4 = t32 @ t43
    H5 = t52

    # Transform the point clouds
    p0 = copy.deepcopy(pcds_all[0]).transform(H0)
    p1 = copy.deepcopy(pcds_all[1]).transform(H1)
    p3 = copy.deepcopy(pcds_all[3]).transform(H3)
    p4 = copy.deepcopy(pcds_all[4]).transform(H4)
    p5 = copy.deepcopy(pcds_all[5]).transform(H5)

    return p0, p1, p3, p4, p5

def main(data_path, tform_path):
    Tforms = load_transformations(tform_path)

    o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Error)
    start_time = time.time()
    pcds_all = load_filter_pcds(data_path, Tforms)
    o3d.visualization.draw_geometries(pcds_all)
    
    p0, p1, p3, p4, p5 = apply_transformations(pcds_all)

    # Combine point clouds
    pcds = o3d.geometry.PointCloud()
    pcds = p0 + p1 + pcds_all[2] + p3 + p4 + p5
    o3d.io.write_point_cloud(data_path+"")

    # Mesh reconstruction
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcds, depth=9)
    sa = mesh.get_surface_area()
    
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"It took {execution_time:.2f}s to reconstruct mesh.")
    print("Surface area is:", sa)

if __name__ == "__main__":
    #parser = argparse.ArgumentParser(description="Process and reconstruct mesh from data.")
    #parser.add_argument("-i", "--input", required=True, help="Path to the data.")
    #parser.add_argument("-t", "--transform", required=True, help="Path to the transformations.")
    #args = parser.parse_args()
    #d = '/home/vigir3d/Datasets/cattle_scans/farm_scan1/Animal_482_2/'
    d = '/home/vigir3d/Datasets/cattle_scans/FirstOne/'
    t = '/home/vigir3d/Datasets/cattle_scans/farm_scan1/Animal_482_1/'
    main(d,t)
    #pcds_all = load_filter_pcds(d, t)
    o3d.visualization.draw_geometries(pcds_all)

# data_path = '/home/vigir3d/Datasets/cattle_scans/farm_scan1/Animal_482_2/'

# tform_path = '/home/vigir3d/Datasets/cattle_scans/farm_scan1/Animal_482_1/'
# h0 = np.loadtxt(tform_path+"htm_0_3.txt")
# h1 = np.loadtxt(tform_path+"htm_1_3.txt")
# h5 = np.loadtxt(tform_path+"htm_5_3.txt")
# h6 = np.loadtxt(tform_path+"htm_6_3.txt")
# h7 = np.loadtxt(tform_path+"htm_7_3.txt")

# Tforms = [h0,h1,h5,h6,h7]


# o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Error)
# start_time = time.time()
# pcds_all = load_filter_pcds(data_path,Tforms);
# t01= colored_ICP(pcds_all[0],pcds_all[1])
# t52= colored_ICP(pcds_all[5],pcds_all[2])
# t43= colored_ICP(pcds_all[4],pcds_all[3])
# t12= colored_ICP(pcds_all[1],pcds_all[2])
# t32= colored_ICP(pcds_all[3],pcds_all[2])


# H0 = t12 @ t01
# H1 = t12
# H3 = t32
# H4 = t32 @ t43
# H5 = t52


# # np.savetxt("c_icp_0_2.txt", H0)
# # np.savetxt("c_icp_1_2.txt", H1)
# # np.savetxt("c_icp_3_2.txt", H3)
# # np.savetxt("c_icp_4_2.txt", H4)
# # np.savetxt("c_icp_5_2.txt", H5)

# p0 = copy.deepcopy(pcds_all[0]).transform(H0)
# p1 = copy.deepcopy(pcds_all[1]).transform(H1)
# p3 = copy.deepcopy(pcds_all[3]).transform(H3)
# p4 = copy.deepcopy(pcds_all[4]).transform(H4)
# p5 = copy.deepcopy(pcds_all[5]).transform(H5)
# #end_time = time.time()
# #execution_time = end_time - start_time
# #print("It took :", execution_time, "s to process all frames")

# # combine pcds

# #print('run Poisson surface reconstruction')

# pcds = o3d.geometry.PointCloud()
# pcds = p0+p1+pcds_all[2]+p3+p4+p5
# #pcds = pcds_all[0]+pcds_all[1]+pcds_all[2]+pcds_all[3]+pcds_all[4]+pcds_all[5]
# with o3d.utility.VerbosityContextManager(
#         o3d.utility.VerbosityLevel.Error) as cm:
#     mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
#         pcds, depth=9)
# sa = mesh.get_surface_area()
# end_time = time.time()
# execution_time = end_time - start_time
# print("It took :", execution_time, "s to reconstruct mesh")
# print(sa)


[2023-09-14 15:36:23.756] [error] [t=15170] /__w/1/s/extern/Azure-Kinect-Sensor-SDK/src/../include/k4ainternal/matroska_read.h (143): k4a_playback_t_get_context(). Invalid k4a_playback_t 0xa4b3cc80
[2023-09-14 15:36:23.756] [error] [t=15170] /__w/1/s/extern/Azure-Kinect-Sensor-SDK/src/record/sdk/playback.cpp (658): Invalid argument to k4a_playback_close(). playback_handle (0xa4b3cc80) is not a valid handle of type k4a_playback_t
[2023-09-14 15:36:28.627] [error] [t=15170] /__w/1/s/extern/Azure-Kinect-Sensor-SDK/src/../include/k4ainternal/matroska_read.h (143): k4a_playback_t_get_context(). Invalid k4a_playback_t 0xc1a9aa0
[2023-09-14 15:36:28.627] [error] [t=15170] /__w/1/s/extern/Azure-Kinect-Sensor-SDK/src/record/sdk/playback.cpp (658): Invalid argument to k4a_playback_close(). playback_handle (0xc1a9aa0) is not a valid handle of type k4a_playback_t
[2023-09-14 15:36:33.435] [error] [t=15170] /__w/1/s/extern/Azure-Kinect-Sensor-SDK/src/../include/k4ainternal/matroska_read.h (143): k4

RuntimeError: [1;31m[Open3D Error] (virtual Eigen::Matrix4d open3d::pipelines::registration::TransformationEstimationForColoredICP::ComputeTransformation(const open3d::geometry::PointCloud&, const open3d::geometry::PointCloud&, const CorrespondenceSet&) const) /root/Open3D/cpp/open3d/pipelines/registration/ColoredICP.cpp:104: No correspondences found between source and target pointcloud.
[0;m

In [ ]:
d = '/home/vigir3d/Datasets/cattle_scans/farm_scan1/Animal_482_2/'

t = '/home/vigir3d/Datasets/cattle_scans/farm_scan1/Animal_482_1/'
#main(d,t)
pcds_all = load_filter_pcds(d, t)
o3d.visualization.draw_geometries(pcds_all)

# Current Pipeline Code

In [ ]:
import cv2
import numpy as np
import open3d as o3d
import json
import glob
import matplotlib.pyplot as plt
import copy
import teaserpp_python
from numpy.linalg import inv
from scipy.spatial import cKDTree
import time
from PIL import Image
import torchvision
import torch
from torchvision import transforms as T 
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from numba import jit, prange
import os
import csv 
# Load MaskRCNN 

from multiprocessing import Pool

def pad_depth_to_color(depth, color):
    # Calculate differences in dimensions
    height_diff = color.shape[0] - depth.shape[0]
    width_diff = color.shape[1] - depth.shape[1]
    
    # Compute padding values for height and width
    pad_top = height_diff // 2
    pad_bottom = height_diff - pad_top
    pad_left = width_diff // 2
    pad_right = width_diff - pad_left
    
    # Apply padding
    padded_depth = np.pad(depth, ((pad_top, pad_bottom), (pad_left, pad_right)), mode='constant', constant_values=0)
    
    return padded_depth

def load_maskrcnn_model(model_path):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, 2)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    return model

def get_model_mask(model_generic, image):
    proba_threshold = 0.5
    ig = transform(image)
    with torch.no_grad():
        prediction = model_generic([ig.to(device)])
        
    if(prediction[0]['masks'].nelement() == 0):
        XX = torch.empty((0,0), dtype=torch.int64)
        return XX
    predicted_mask = prediction[0]
    predicted_mask = predicted_mask['masks'][0] > proba_threshold
    
    predicted_mask = predicted_mask.squeeze(1)
    mask = predicted_mask.cpu().detach()
    return mask


def segment_images_modified(last_frame):
    depth_PIL = Image.fromarray(np.asarray(last_frame.depth)).convert("RGB")
    rgb_image_array = np.asarray(last_frame.color)
    depth_image_array = np.asarray(last_frame.depth)
    rgb_PIL = Image.fromarray(rgb_image_array)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    rgb_mask = get_model_mask(model_rgb, rgb_PIL)
    depth_mask = get_model_mask(model_depth, depth_PIL)

    if depth_mask.nelement() == 0:
        mask_combined = rgb_mask
    else:
        mask_combined = depth_mask | rgb_mask  # 1 vote arbitration (OR the masks)

    # Convert tensor to numpy array and ensure the right datatype
    mask_combined = mask_combined.numpy().astype(rgb_image_array.dtype)
    mask_image = mask_combined.swapaxes(0, 2).swapaxes(0, 1)
    mask_image = (mask_image > 0).astype(rgb_image_array.dtype)   

    fg_image_rgb = rgb_image_array * mask_image

    # For the depth image:
    squeezed_mask = np.squeeze(mask_image)
    fg_image_depth = depth_image_array * squeezed_mask
    
    
    last_frame.color = o3d.geometry.Image(fg_image_rgb)
    last_frame.depth = o3d.geometry.Image(fg_image_depth)
   


    return last_frame


def filter_file_names(file_list): # to avoid the two head cameras
    new_file_list = []
    for f in file_list:
        tmp = f.split('.mkv')[0]
        ID  = tmp.split('_')[-1]
        if (ID == "13" or ID == "15"):
            continue
        else :
            new_file_list.append(f)
    return new_file_list


def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

def cluster_point_cloud_new(outlier_cloud):
    cloud_colors = copy.deepcopy(np.asarray(outlier_cloud.colors).T)
    
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        labels = np.array(outlier_cloud.cluster_dbscan(eps=0.1, min_points=10, print_progress=False))

    # Identify the largest cluster
    values, counts = np.unique(labels, return_counts=True)
    ind = np.argmax(counts)
    largest_cluster_label = values[ind]
    #print(f"Largest cluster label: {largest_cluster_label}")

    # Filter points, normals, and colors for the largest cluster
    cloud_xyz = pcd2xyz(outlier_cloud)
    cloud_normals = pcd2normals(outlier_cloud)

    cloud_filtered = cloud_xyz[:, labels == largest_cluster_label]
    normals_filtered = cloud_normals[:, labels == largest_cluster_label]
    colors_filtered = cloud_colors[:, labels == largest_cluster_label]

    # Create a point cloud for the largest cluster
    pcd_filtered_largest_cluster = o3d.geometry.PointCloud()
    pcd_filtered_largest_cluster.points = o3d.utility.Vector3dVector(cloud_filtered.T)
    pcd_filtered_largest_cluster.normals = o3d.utility.Vector3dVector(normals_filtered.T)
    pcd_filtered_largest_cluster.colors = o3d.utility.Vector3dVector(colors_filtered.T)

    #o3d.visualization.draw_geometries([pcd_filtered_largest_cluster])
    return pcd_filtered_largest_cluster

def upsample_using_reference_normals(sparse_pcd, dense_pcd, search_radius=0.02, angle_threshold=30):
    """
    Efficiently upsample the sparse point cloud using the dense point cloud as reference.
    Considers the normals to ensure added points are consistent with the sparse cloud.

    Parameters:
    - sparse_pcd: The sparse point cloud
    - dense_pcd: The dense reference point cloud
    - search_radius: Radius to search for neighbors
    - angle_threshold: Maximum angle in degrees between normals to consider a point

    Returns:
    - A new upsampled point cloud
    """
    # Ensure the point clouds have normals
    if not sparse_pcd.has_normals():
        sparse_pcd.estimate_normals()
    if not dense_pcd.has_normals():
        dense_pcd.estimate_normals()

    kdtree = o3d.geometry.KDTreeFlann(dense_pcd)

    sparse_points = np.asarray(sparse_pcd.points)
    sparse_normals = np.asarray(sparse_pcd.normals)
    added_points = set(map(tuple, sparse_points))

    upsampled_points = list(sparse_points)

    for i, point in enumerate(sparse_points):
        [k, idx, _] = kdtree.search_radius_vector_3d(point, search_radius)
        
        neighbors = np.asarray(dense_pcd.points)[idx]
        neighbor_normals = np.asarray(dense_pcd.normals)[idx]

        # Calculate angles between sparse point's normal and all its neighbors' normals
        angles = np.degrees(np.arccos(np.clip(np.dot(sparse_normals[i], neighbor_normals.T), -1.0, 1.0)))

        # Filtering neighbors based on angle threshold
        valid_neighbors = neighbors[angles < angle_threshold]

        # Filtering out points that are already in the sparse cloud or have been added before
        unique_valid_neighbors = [tuple(neighbor) for neighbor in valid_neighbors if tuple(neighbor) not in added_points]

        upsampled_points.extend(unique_valid_neighbors)
        added_points.update(unique_valid_neighbors)

    upsampled_pcd = o3d.geometry.PointCloud()
    upsampled_pcd.points = o3d.utility.Vector3dVector(upsampled_points)
    upsampled_pcd.estimate_normals()

    return upsampled_pcd


def pcd2normals(pcd):
    return np.asarray(pcd.normals).T
def pcd2xyz(pcd):
    return np.asarray(pcd.points).T


@jit(nopython=True, parallel=True)
def numba_eliminate_flying_pixels(depth_image, ws, threshold):
    height, width = depth_image.shape
    result = np.zeros_like(depth_image, dtype=np.float64)
    
    for cy in prange(height):
        for cx in prange(width):
            x_start, x_end = max(0, cx - ws), min(width, cx + ws + 1)
            y_start, y_end = max(0, cy - ws), min(height, cy + ws + 1)
            window = depth_image[y_start:y_end, x_start:x_end]
            result[cy, cx] = np.sum(np.abs(window - depth_image[cy, cx]))
    
    return result

def colored_ICP(source, target):
    
    voxel_radius = [0.04, 0.02, 0.01]
    max_iter = [50, 30, 14]
    current_transformation = np.identity(4)
    #print("3. Colored point cloud registration")
    for scale in range(3):
        iters = max_iter[scale]
        radius = voxel_radius[scale]
        #print("iteration: ", iters, radius, scale)

        #print("3-1. Downsample with a voxel size %.2f" % radius)
        source_down = copy.deepcopy(source).voxel_down_sample(radius)
        target_down = copy.deepcopy(target).voxel_down_sample(radius)

        #print("3-2. Estimate normal.")
        source_down.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
        target_down.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))

        #print("3-3. Applying colored point cloud registration")
        result_icp = o3d.pipelines.registration.registration_colored_icp(
            source_down, target_down, radius, current_transformation,
            o3d.pipelines.registration.TransformationEstimationForColoredICP(),
            o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                              relative_rmse=1e-6,
                                                              max_iteration=iters))
        current_transformation = result_icp.transformation
    
   
        #draw_registration_result(source, target, current_transformation)
    return current_transformation

def backproject_o3d(rgbd_frame, intrinsics):
    
    rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        rgbd_frame.color, rgbd_frame.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbdc, intrinsics)
    n_radius = 0.01*2.0
    pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=n_radius, max_nn=30))
    
    return pcd

def load_calibration(tform_path, num_transforms=5):
    transforms = [np.eye(4)]  # Initialize with identity matrix
    
    # Load transformations from file
    for i in range(1, num_transforms+1):
        filename = tform_path + f"H_0_{i}.txt"
        
        if not os.path.exists(filename):
            raise FileNotFoundError(f"The file {filename} does not exist!")
        
        transforms.append(np.loadtxt(filename))
    
    return transforms

def perform_pairwise_alignment(pcds_tsdf,pcds_cropped):
    #"""Compute and apply transformations."""
    t01 = colored_ICP(pcds_tsdf[0], pcds_tsdf[1])
    t52 = colored_ICP(pcds_tsdf[5], pcds_tsdf[2])
    t43 = colored_ICP(pcds_tsdf[4], pcds_tsdf[3])
    t12 = colored_ICP(pcds_tsdf[1], pcds_tsdf[2])
    t32 = colored_ICP(pcds_tsdf[3], pcds_tsdf[2])

    H0 = t12 @ t01
    H1 = t12
    H3 = t32
    H4 = t32 @ t43
    H5 = t52

    # Transform the point clouds
    p0 = copy.deepcopy(pcds_cropped[0]).transform(H0)
    p1 = copy.deepcopy(pcds_cropped[1]).transform(H1)
    p3 = copy.deepcopy(pcds_cropped[3]).transform(H3)
    p4 = copy.deepcopy(pcds_cropped[4]).transform(H4)
    p5 = copy.deepcopy(pcds_cropped[5]).transform(H5)
    
    d0 = copy.deepcopy(pcds_tsdf[0]).transform(H0)
    d1 = copy.deepcopy(pcds_tsdf[1]).transform(H1)
    d3 = copy.deepcopy(pcds_tsdf[3]).transform(H3)
    d4 = copy.deepcopy(pcds_tsdf[4]).transform(H4)
    d5 = copy.deepcopy(pcds_tsdf[5]).transform(H5)
    
    pcd_combined = o3d.geometry.PointCloud()
    pcd_combined = p0+p1+pcds_cropped[2]+p3+p4+p5
    
    ptsdf_combined = o3d.geometry.PointCloud()
    ptsdf_combined = d0+d1+pcds_tsdf[2]+d3+d4+d5

    return pcd_combined, ptsdf_combined

def process_file(i):
    inFile = files[i]
    fname = inFile.split('/')[-1]
    file_name = fname.split('.mkv')[0]
    
    reader = o3d.io.AzureKinectMKVReader()
    reader.open(inFile)
    
    if not reader.is_opened():
        raise RuntimeError("Unable to open file {}".format(inFile))
    
    metadata = reader.get_metadata()

    # write the metadata to a JSON file since that seems to be the only
    # way to retrieve that data
    o3d.io.write_azure_kinect_mkv_metadata('{}/{}_intrinsic.json'.format(abspath, file_name), metadata)

    # Open the file and load the JSON
    with open(abspath + "/" + file_name + "_intrinsic.json") as f:
        data = json.load(f)
    
    height = data['height']
    width = data['width']
    intrinsics = data["intrinsic_matrix"]
    time_stamp = data["stream_length_usec"]

    camera_intrinsics = o3d.camera.PinholeCameraIntrinsic()
    cx = intrinsics[6]
    cy = intrinsics[7]
    fx = intrinsics[0]
    fy = intrinsics[4]
    camera_intrinsics.set_intrinsics(width, height, fx, fy, cx, cy)

    last_frame = None
    while not reader.is_eof():
        rgbda = reader.next_frame()
        if rgbda is None:
            continue
        last_frame = rgbda

    if last_frame is not None:
        return last_frame
    else:
        print("************No valid frames found in the .mkv file.**********")
        return None

def upsample_using_reference_normals_new(sparse_pcd, dense_pcd, search_radius=0.02, angle_threshold=30):
    # Ensure the point clouds have normals
    if not sparse_pcd.has_normals():
        sparse_pcd.estimate_normals()
    if not dense_pcd.has_normals():
        dense_pcd.estimate_normals()

    dense_points = np.asarray(dense_pcd.points)
    dense_tree = cKDTree(dense_points)

    sparse_points = np.asarray(sparse_pcd.points)
    sparse_normals = np.asarray(sparse_pcd.normals)
    
    # This retrieves the indices of all neighbors within the search_radius
    neighbor_indices = dense_tree.query_ball_point(sparse_points, search_radius, workers=-1)
    
    valid_indices = []
    for i, idx_row in enumerate(neighbor_indices):
        neighbors = dense_points[idx_row]
        neighbor_normals = np.asarray(dense_pcd.normals)[idx_row]

        # Calculate angles between sparse point's normal and all its neighbors' normals
        angles = np.degrees(np.arccos(np.clip(np.dot(sparse_normals[i], neighbor_normals.T), -1.0, 1.0)))

        # Filtering neighbors based on angle threshold
        valid_neighbor_indices = np.array(idx_row)[angles < angle_threshold]
        valid_indices.extend(valid_neighbor_indices)
#     valid_indices = []
#     for i, idx_row in enumerate(neighbor_indices):
#         neighbors = dense_points[idx_row]
#         neighbor_normals = np.asarray(dense_pcd.normals)[idx_row]
        
#         # Calculate angles between sparse point's normal and all its neighbors' normals
#         angles = np.degrees(np.arccos(np.clip(np.dot(sparse_normals[i], neighbor_normals.T), -1.0, 1.0)))

#         # Filtering neighbors based on angle threshold
#         valid_for_current = idx_row[angles < angle_threshold]
#         valid_indices.extend(valid_for_current)

    unique_valid_indices = np.unique(valid_indices)
    final_valid_neighbors = dense_points[unique_valid_indices]
    
    upsampled_points = np.vstack([sparse_points, final_valid_neighbors])

    upsampled_pcd = o3d.geometry.PointCloud()
    upsampled_pcd.points = o3d.utility.Vector3dVector(upsampled_points)
    upsampled_pcd.estimate_normals()

    return upsampled_pcd


def load_filter_pcds(data_path, ctform):  # returns a list of  pcds
    ws = 2
    flying_pixel_filter_threshold = 1000

    reader = o3d.io.AzureKinectMKVReader()
    abspath = data_path
    pcds = []
    files_raw = glob.glob(data_path+'/*.mkv')
    files = filter_file_names(files_raw)
    files.sort()
   
    list_size = len(files)
    rgbd_frames = [None] * list_size
    pcds_tsdf = []
    
    l1 = time.time()
    for i in range(len(files)): # for each view
   
        inFile = files[i]
        fname = inFile.split('/')[-1]
        file_name = fname.split('.mkv')[0]
        reader.open(inFile)
        if not reader.is_opened():
            raise RuntimeError("Unable to open file {}".format(inFile))
        metadata = reader.get_metadata()
  
        # write the metadata to a JSON file since that seems to be the only
        # way to retrieve that data
        o3d.io.write_azure_kinect_mkv_metadata(
                    '{}/{}_intrinsic.json'.format(abspath,file_name), metadata)

        # Open the file and load the JSON
        with open(abspath+"/" + file_name + "_intrinsic.json") as f:
            data = json.load(f)
        
        height = data['height']
        width = data['width']
        intrinsics = data["intrinsic_matrix"]
        time_stamp = data["stream_length_usec"]
        #print(f"Intrinsic Matrix {intrinsics}")

        camera_intrinsics = o3d.camera.PinholeCameraIntrinsic()
        cx = intrinsics[6]
        cy = intrinsics[7]
        fx = intrinsics[0]
        fy = intrinsics[4]
        camera_intrinsics.set_intrinsics(width,height,fx,fy,cx,cy)

        last_frame = None
        while not reader.is_eof(): # go until hitting eof because of exposure issues in early color frames
            rgbda = reader.next_frame();
            if rgbda is None:
                #print("Got nothing! ")
                continue
            last_frame = rgbda

        if last_frame is not None:
            #print("Got the last frame")
            rgbd_frames[i] = last_frame
        else:
            
            print("************No valid frames found in the .mkv file.**********")
    
        # filter the depth image for flying pixels
        depth_image_array = np.asarray(last_frame.depth)
        rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        last_frame.color, last_frame.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
        volume = o3d.pipelines.integration.ScalableTSDFVolume(
                    voxel_length= 4.0/ 512.0,
                    sdf_trunc=0.4,
                    color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)

        volume.integrate(
            rgbdc, # rgbdc originally - trying with masked_rgbd
            camera_intrinsics,
            np.eye(4),
        )
#         pcd_raw = backproject_o3d(last_frame,camera_intrinsics)
        pcd_tsdf = volume.extract_point_cloud()
        # First eliminate flying pixels before integrating the depth map
        result_mask = numba_eliminate_flying_pixels(depth_image_array.copy(), ws, flying_pixel_filter_threshold)
        depth_image_array[result_mask > flying_pixel_filter_threshold] = 0
        # Re-insert filtered depth into the rgbd image
        last_frame.depth = o3d.geometry.Image(depth_image_array)
      
        # Apply maskrcnn to filtered depth image
        masked_rgbd = segment_images_modified(last_frame,i,data_path)
        # necessary lol
        rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        masked_rgbd.color, masked_rgbd.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
        pcd_tmp = o3d.geometry.PointCloud.create_from_rgbd_image(rgbdc,
                                                            camera_intrinsics)
      
        #print("Time was: ", w2-w1, " s")
        pcds.append(copy.deepcopy(pcd_tmp).transform(ctform[i]))
        pcds_tsdf.append(copy.deepcopy(pcd_tsdf).transform(ctform[i]))
        reader.close()
    l2 = time.time()  
    print("Loading files: ", l2-l1, " s")
    return pcds,pcds_tsdf


def save_results(animal_id, data_path, mesh, pcd_downsampled, sa, v):
    # Save the mesh and PCD
    mesh_filename = os.path.join(data_path, f"{animal_id}_mesh.ply")
    pcd_filename = os.path.join(data_path, f"{animal_id}_pcd_downsampled.ply")

    o3d.io.write_triangle_mesh(mesh_filename, mesh)
    
    o3d.io.write_point_cloud(pcd_filename, pcd_downsampled)

    CSV_PATH = '/home/vigir3d/Datasets/cattle_scans/cow_measurements.csv'
    # Check if CSV file exists to decide whether to write headers
    write_header = not os.path.exists(CSV_PATH)
  # Save SA & V to the CSV file in append mode
    with open(CSV_PATH, 'a', newline='') as csvfile:
        fieldnames = ['Animal ID', 'SA', 'V']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        if write_header:
            writer.writeheader()

        writer.writerow({'Animal ID': animal_id, 'SA': sa, 'V': v})
        
def cluster_point_cloud_tensor(pcd) :
    colors = pcd.point.colors
    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Error) as cm:
        labels = pcd.cluster_dbscan(eps=0.1, min_points=10, print_progress=False)
        
    
    labels = labels.cpu()
    # Identify the largest cluster
    values, counts = np.unique(labels, return_counts=True)
    ind = np.argmax(counts)
    largest_cluster_label = values[ind]
    #print(f"Largest cluster label: {largest_cluster_label}")

    colors[labels < 0] = 0
    pcd.point.colors = colors
    return pcd


# Main
rgb_model_path = '/home/vigir3d/Datasets/cattle_scans/maskrcnn_data/maskrcnn_v2.pth'
depth_model_path = '/home/vigir3d/Datasets/cattle_scans/maskrcnn_data/maskrcnn_depth_best.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loads the maskrcnn trained for depth and rgb
model_rgb = load_maskrcnn_model(rgb_model_path)
model_depth = load_maskrcnn_model(depth_model_path)


transform = T.ToTensor()
# These two will need to be requested from command line
# -i   | input data path
# - t  | calibration data path 
data_path = '/home/vigir3d/Datasets/cattle_scans/Cattle_scan_11_17_22/Animal_5_2'
tform_path = '/home/vigir3d/Datasets/cattle_scans/farm_07_24/Animal_calib/'

if __name__ == "__main__":
    # Run things
    tensor = False
    if  tensor :
        s = time.time()

        l1 = time.time()
        tforms = load_calibration(tform_path,5)
        pcds, ptsdf = load_filter_pcds(data_path,tforms)
        pcd_all, ptsdf_all = perform_pairwise_alignment(ptsdf,pcds)
        l2 = time.time()

        #place on the GPU
        # Tensorize
        # get the Device
        p1 = time.time()
        o3d_device = o3d.core.Device("CUDA:0") 
        # convert segmented and full clouds to tensor
        pcd_t = o3d.t.geometry.PointCloud.from_legacy(pcd_all).to(o3d_device)
        #pcd_t_tsdf = o3d.t.geometry.PointCloud.from_legacy(ptsdf_all).to(o3d_device)
        pcd_t.estimate_normals()
        # Perform clustering to get largest cloud of points from segmented cloud
        pcd_clustered = cluster_point_cloud_tensor(pcd_t)
        # place on cpu
        pcd_clustered_cpu = pcd_clustered.cpu().to_legacy()
        p2 = time.time()

        u1 = time.time()
        # Upsample the single cluster cloud using the full cloud as reference
        pcd_upsampled = upsample_using_reference_normals_new(pcd_clustered_cpu, ptsdf_all)
        u2 = time.time()

        tn1 = time.time()
        # Downsample uniformly to ease compute
        pcd_downsampled = pcd_upsampled.voxel_down_sample(0.01)
        # Correct for the normal orientation problem
        pcd_downsampled.orient_normals_consistent_tangent_plane(30)
        tn2 = time.time()

        m1 = time.time()
        with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
                 mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd_downsampled, depth=6)
        m2 = time.time()
        sa = mesh.get_surface_area()
        if(mesh.is_watertight()):
            #print("Is watertight 1")
            v =  mesh.get_volume()
        else : 
            v = 0.0
        animal_id = os.path.basename(os.path.normpath(data_path))
        save_results(animal_id, data_path, mesh, pcd_downsampled, sa, v)
        e = time.time()

        print("Total time: ", e-s, " s")


        print("Load and register: ", l2-l1, " s")

        print("Preprocess: ", p2-p1, " s")

        print("Upsampling: ", u2-u1, " s")
        print("Normal orientation: ", tn2-tn1, " s")    
        print("Meshing: ", m2-m1, " s")
    
    elif tensor :
        # Run things
        s = time.time()

        l1 = time.time()
        tforms = load_calibration(tform_path,5)
        pcds, ptsdf = load_filter_pcds(data_path,tforms)
        l2 = time.time()
        pcd_all, ptsdf_all = perform_pairwise_alignment(ptsdf,pcds)
        r2 = time.time()
        
        p1 = time.time()
        pcd_all.estimate_normals()
        # cluster first to remove extra noise
        pcd_clustered = cluster_point_cloud_new(pcd_all)
        # After clustering, we upsample from the initial
        p2 = time.time()

        u1 = time.time()
        pcd_upsampled = upsample_using_reference_normals_new(pcd_clustered, ptsdf_all)
        u2 = time.time()

        tn1 = time.time()
        pcd_downsampled = pcd_upsampled.voxel_down_sample(0.01)
        #pcd_downsampled.orient_normals_towards_camera_location()
        pcd_downsampled.orient_normals_consistent_tangent_plane(30)
        tn2 = time.time()

        m1 = time.time()
        with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
                mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd_downsampled, depth=6)

        sa = mesh.get_surface_area()
        if(mesh.is_watertight()):
            #print("Is watertight 1")
            v =  mesh.get_volume()
        else : 
            v = 0.0
        m2 = time.time()
        animal_id = os.path.basename(os.path.normpath(data_path))
        save_results(animal_id, data_path, mesh, pcd_downsampled, sa, v)

        e = time.time()
        print("Load files: ", l2-l1, " s")
        print("Register clouds: ", r2-l2, " s")

        print("Remove extra points: ", p2-p1, " s")

        print("Upsampling: ", u2-u1, " s")
        print("Normal orientation: ", tn2-tn1, " s")    
        print("Meshing: ", m2-m1, " s")
        print("Total time: ", e-s, " s")


# Non tensor version

In [ ]:
# Run things
data_path = '/home/vigir3d/Datasets/cattle_scans/Cattle_scan_11_17_22/Animal_5_2'
tform_path = '/home/vigir3d/Datasets/cattle_scans/farm_07_24/Animal_calib/'
s = time.time()

l1 = time.time()
tforms = load_calibration(tform_path,5)
pcds, ptsdf = load_filter_pcds(data_path,tforms)
pcd_all, ptsdf_all = perform_pairwise_alignment(ptsdf,pcds)
l2 = time.time()

p1 = time.time()
pcd_all.estimate_normals()
# cluster first to remove extra noise
pcd_clustered = cluster_point_cloud_new(pcd_all)
# After clustering, we upsample from the initial
p2 = time.time()

u1 = time.time()
pcd_upsampled = upsample_using_reference_normals_new(pcd_clustered, ptsdf_all)
u2 = time.time()

tn1 = time.time()
pcd_downsampled = pcd_upsampled.voxel_down_sample(0.01)
#pcd_downsampled.orient_normals_towards_camera_location()
pcd_downsampled.orient_normals_consistent_tangent_plane(30)
tn2 = time.time()

m1 = time.time()
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd_downsampled, depth=6)

sa = mesh.get_surface_area()
if(mesh.is_watertight()):
    #print("Is watertight 1")
    v =  mesh.get_volume()
else : 
    v = 0.0
m2 = time.time()
animal_id = os.path.basename(os.path.normpath(data_path))
save_results(animal_id, data_path, mesh, pcd_downsampled, sa, v)

e = time.time()

print("Total time: ", e-s, " s")


print("Load and register: ", l2-l1, " s")

print("Preprocess: ", p2-p1, " s")

print("Upsampling: ", u2-u1, " s")
print("Normal orientation: ", tn2-tn1, " s")    
print("Meshing: ", m2-m1, " s")

# Playing with the tensor class

In [ ]:
import open3d.core as o3c

def load_filter_pcds_tensor(data_path, ctform):  # returns a list of  pcds
    ws = 2
    flying_pixel_filter_threshold = 1000

    reader = o3d.io.AzureKinectMKVReader()
    abspath = data_path
    pcds = []
    files_raw = glob.glob(data_path+'/*.mkv')
    files = filter_file_names(files_raw)
    files.sort()
   
    list_size = len(files)
    rgbd_frames = [None] * list_size
    pcds_tsdf = []
    
    
    for i in range(len(files)): # for each view
   
        inFile = files[i]
        fname = inFile.split('/')[-1]
        file_name = fname.split('.mkv')[0]
        print(file_name)
        reader.open(inFile)
        if not reader.is_opened():
            raise RuntimeError("Unable to open file {}".format(inFile))
        metadata = reader.get_metadata()
  
        # write the metadata to a JSON file since that seems to be the only
        # way to retrieve that data
        o3d.io.write_azure_kinect_mkv_metadata(
                    '{}/{}_intrinsic.json'.format(abspath,file_name), metadata)

        # Open the file and load the JSON
        with open(abspath+"/" + file_name + "_intrinsic.json") as f:
            data = json.load(f)
        
        height = data['height']
        width = data['width']
        intrinsics = data["intrinsic_matrix"]
        time_stamp = data["stream_length_usec"]
        #print(f"Intrinsic Matrix {intrinsics}")

        camera_intrinsics = o3d.camera.PinholeCameraIntrinsic()
        cx = intrinsics[6]
        cy = intrinsics[7]
        fx = intrinsics[0]
        fy = intrinsics[4]
        camera_intrinsics.set_intrinsics(width,height,fx,fy,cx,cy)

        last_frame = None
        while not reader.is_eof(): # go until hitting eof because of exposure issues in early color frames
            rgbda = reader.next_frame();
            if rgbda is None:
                #print("Got nothing! ")
                continue
            last_frame = rgbda

        if last_frame is not None:
            #print("Got the last frame")
            rgbd_frames[i] = last_frame
        else:
            
            print("************No valid frames found in the .mkv file.**********")
        
        cname = data_path+"/"+file_name+"_color.png"
        dname = data_path+"/"+file_name+"_depth.jpg"
        o3d.io.write_image(cname, last_frame.color)
        o3d.io.write_image(dname, last_frame.depth)
        
        
        color = o3d.t.io.read_image(cname).to(
        o3d.core.Device("CUDA:0")
        )
        depth = o3d.t.io.read_image(dname).to(
            o3d.core.Device("CUDA:0")
        )
        
#         # filter the depth image for flying pixels
#         cimage_tensor = o3d.core.Tensor.from_numpy(np.asarray(last_frame.color))
#         dimage_tensor = o3d.core.Tensor.from_numpy(np.asarray(last_frame.depth))        
        
#         # Create an Open3D tensor image
#         color_image = o3d.t.geometry.Image(cimage_tensor).to( o3d.core.Device("CUDA:0"))
#         depth_image = o3d.t.geometry.Image(dimage_tensor).to( o3d.core.Device("CUDA:0"))
#         #bf_depth_image = o3d.t.geometry.Image.filter_bilateral(depth_image,3,20,10)
        #rgbd_t = o3d.t.geometry.RGBDImage(color_image,bf_depth_image, True)
        
        K = o3d.core.Tensor(camera_intrinsics.intrinsic_matrix,)
        extrinsics = o3d.core.Tensor(np.linalg.inv(np.eye(4)),)
        # integrate on the GPU
        device = o3d.core.Device('CUDA:0')
       
        
        volume = o3d.t.geometry.VoxelBlockGrid(
            attr_names=("tsdf", "weight", "color"),
            attr_dtypes=(o3c.float32, o3c.float32, o3c.float32),
            attr_channels=((1), (1), (3)),
            voxel_size=3.0 / 512,
            block_resolution=16,
            block_count=50000,
            device=o3d.core.Device("CUDA:0"),
        )

        
        depth_max = 4
        depth_scale = 1000.0
        
        frustum_block_coords = volume.compute_unique_block_coordinates(
            depth, K, extrinsics, depth_max=4.0
            )

        volume.integrate(
            frustum_block_coords,
            depth,
            color,
            K,
            extrinsics,
            depth_max=4.0,
            )
        pcd = volume.extract_point_cloud()
        print(pcd)
        p = pcd.to_legacy()
        #o3d.visualization.draw_geometries([pcd.to_legacy()])

           
        #vbg.integrate(frustum_block_coords, depth_image, color_image, K, K, ext, depth_scale, depth_max)
        
        
        #pcd = vbg.extract_point_cloud()
        #o3d.visualization.draw([pcd])
        
        print("Done Integrating ...!")

        pcds.append(p)
        
        
        
        
        
    return pcds,pcds_tsdf


In [ ]:
data_path = '/home/vigir3d/Datasets/cattle_scans/Cattle_scan_11_17_22/Animal_5_2'
tform_path = '/home/vigir3d/Datasets/cattle_scans/farm_07_24/Animal_calib/'
tforms = load_calibration(tform_path,5)

s1 = time.time()
p1,t1 = load_filter_pcds_tensor(data_path,tforms)
s2 = time.time()

print("Duration is: ", s2-s1)


s3 = time.time()
p2,t2 = load_filter_pcds(data_path,tforms)
s4 = time.time()

print("Duration is: ", s4-s3)

In [ ]:
p

In [ ]:
import multiprocessing

def process_file(i):
    #ws = 2
    #flying_pixel_filter_threshold = 1000
    #print(multiaprocessing.current_process())
    inFile = files[i]
    fname = inFile.split('/')[-1]
    file_name = fname.split('.mkv')[0]
    
    reader = o3d.io.AzureKinectMKVReader()
    reader.open(inFile)
    
    if not reader.is_opened():
        raise RuntimeError("Unable to open file {}".format(inFile))
    
    metadata = reader.get_metadata()

    # write the metadata to a JSON file since that seems to be the only
    # way to retrieve that data
    o3d.io.write_azure_kinect_mkv_metadata('{}/{}_intrinsic.json'.format(abspath, file_name), metadata)

    # Open the file and load the JSON
    with open(abspath + "/" + file_name + "_intrinsic.json") as f:
        data = json.load(f)
    
    height = data['height']
    width = data['width']
    intrinsics = data["intrinsic_matrix"]
    time_stamp = data["stream_length_usec"]

    camera_intrinsics = o3d.camera.PinholeCameraIntrinsic()
    cx = intrinsics[6]
    cy = intrinsics[7]
    fx = intrinsics[0]
    fy = intrinsics[4]
    camera_intrinsics.set_intrinsics(width, height, fx, fy, cx, cy)
    #K = np.array([width,height,fx,fy,cx,cy])
    last_frame = None
    while not reader.is_eof():
        rgbda = reader.next_frame()
        if rgbda is None:
            continue
        last_frame = rgbda

    if last_frame is not None:
        #print("************No valid frames found in the .mkv file.**********")
    
        print("Got last_frame")
        #return last_frame
    else:
        print("************No valid frames found in the .mkv file.**********")
        #return None
    
    reader.close()
    rgbd_frames_glob[i] = last_frame
    camera_intrinsics_glob[i] = camera_intrinsics
    #return [np.asarray(last_ = frame.color), np.asarray(last_frame.depth), K]



def load_filter_pcds_par(data_path, ctform):  # returns a list of  pcds
    ws = 2
    flying_pixel_filter_threshold = 1000

    reader = o3d.io.AzureKinectMKVReader()
    pcds = []
    files_raw = glob.glob(data_path+'/*.mkv')
    files = filter_file_names(files_raw)
    files.sort()
   
    list_size = len(files)
    rgbd_frames = [None] * list_size
    pcds_tsdf = []
    # Define the number of processes you want to spawn

    NUM_PROCESSES =4
    with Pool(processes=NUM_PROCESSES) as pool:
        pool.map(process_file, range(len(files)))
        
    
    print("Returned: ", len(rgbd_frames_glob))
    for i in range(len(rgbd_frames_glob)):
       
        last_frame = rgbd_frames_glob[i]
        camera_intrinsics = camera_intrinsics_glob[i]
        depth_image_array = np.asarray(last_frame.depth)

        rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        last_frame.color, last_frame.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
      
        volume = o3d.pipelines.integration.ScalableTSDFVolume(
                    voxel_length= 4.0/ 512.0,
                    sdf_trunc=0.4,
                    color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)

        volume.integrate(
            rgbdc, # rgbdc originally - trying with masked_rgbd
            camera_intrinsics,
            np.eye(4),
        )
    #         pcd_raw = backproject_o3d(last_frame,camera_intrinsics)
        pcd_tsdf = volume.extract_point_cloud()
        # First eliminate flying pixels before integrating the depth map
        result_mask = numba_eliminate_flying_pixels(depth_image_array.copy(), ws, flying_pixel_filter_threshold)
        depth_image_array[result_mask > flying_pixel_filter_threshold] = 0
        # Re-insert filtered depth into the rgbd image
        last_frame.depth = o3d.geometry.Image(depth_image_array)

        # Apply maskrcnn to filtered depth image
        masked_rgbd = segment_images_modified(last_frame)
        # necessary lol
        rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        masked_rgbd.color, masked_rgbd.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
        pcd_tmp = o3d.geometry.PointCloud.create_from_rgbd_image(rgbdc,
                                                            camera_intrinsics)
        pcds.append(pcd_tmp)
        pcds_tsdf.append(pcd_tsdf)


    return pcds,pcds_tsdf

In [ ]:
def load_filter_pcds_par(data_path, ctform):  # returns a list of  pcds
    ws = 2
    flying_pixel_filter_threshold = 1000

    reader = o3d.io.AzureKinectMKVReader()
    pcds = []
    files_raw = glob.glob(data_path+'/*.mkv')
    files = filter_file_names(files_raw)
    files.sort()
   
    list_size = len(files)
    rgbd_frames = [None] * list_size
    pcds_tsdf = []
    # Define the number of processes you want to spawn

    NUM_PROCESSES =4
    with Pool(processes=NUM_PROCESSES) as pool:
        pool.map(process_file, range(len(files)))
        
    
    print("Returned: ", len(rgbd_frames_glob))
    for i in range(len(rgbd_frames_glob)):
       
        last_frame = rgbd_frames_glob[i]
        camera_intrinsics = camera_intrinsics_glob[i]
        depth_image_array = np.asarray(last_frame.depth)

        rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        last_frame.color, last_frame.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
      
        volume = o3d.pipelines.integration.ScalableTSDFVolume(
                    voxel_length= 4.0/ 512.0,
                    sdf_trunc=0.4,
                    color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)

        volume.integrate(
            rgbdc, # rgbdc originally - trying with masked_rgbd
            camera_intrinsics,
            np.eye(4),
        )
    #         pcd_raw = backproject_o3d(last_frame,camera_intrinsics)
        pcd_tsdf = volume.extract_point_cloud()
        # First eliminate flying pixels before integrating the depth map
        result_mask = numba_eliminate_flying_pixels(depth_image_array.copy(), ws, flying_pixel_filter_threshold)
        depth_image_array[result_mask > flying_pixel_filter_threshold] = 0
        # Re-insert filtered depth into the rgbd image
        last_frame.depth = o3d.geometry.Image(depth_image_array)

        # Apply maskrcnn to filtered depth image
        masked_rgbd = segment_images_modified(last_frame)
        # necessary lol
        rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        masked_rgbd.color, masked_rgbd.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
        pcd_tmp = o3d.geometry.PointCloud.create_from_rgbd_image(rgbdc,
                                                            camera_intrinsics)
        pcds.append(pcd_tmp)
        pcds_tsdf.append(pcd_tsdf)


    return pcds,pcds_tsdf

In [ ]:
for i in range(len(pcds)):
    o3d.visualization.draw_geometries([ptsdf1[i]])

In [ ]:
o3d.visualization.draw_geometries(pcds1)

In [ ]:
tform_path = '/home/vigir3d/Datasets/cattle_scans/farm_07_24/Animal_calib/'
tforms = load_calibration(tform_path,5)

files_raw = glob.glob(data_path+'/*.mkv')
abspath = data_path
files = filter_file_names(files_raw)
files.sort()

list_size = len(files)
rgbd_frames_glob = [None] * list_size
camera_intrinsics_glob = [None] * list_size
NUM_PROCESSES =4
with Pool(processes=NUM_PROCESSES) as pool:
    pool.map(process_file, range(len(files)))
s1 = time.time()
pcds, ptsdf = load_filter_pcds_par(data_path,tforms)
s2 = time.time()


s3 = time.time()
pcds1, ptsdf1 = load_filter_pcds(data_path,tforms)
s4 = time.time()

print("par took: ", s2-s1, "s")
print("regular took: ", s4-s3, "s")

# This works!!! 5:19AM

In [ ]:
import concurrent.futures
def process_file(i):
    #ws = 2
    #flying_pixel_filter_threshold = 1000
    #print(multiprocessing.current_process())
    inFile = files[i]
    fname = inFile.split('/')[-1]
    file_name = fname.split('.mkv')[0]
    
    reader = o3d.io.AzureKinectMKVReader()
    reader.open(inFile)
    
    if not reader.is_opened():
        raise RuntimeError("Unable to open file {}".format(inFile))
    
    metadata = reader.get_metadata()

    # write the metadata to a JSON file since that seems to be the only
    # way to retrieve that data
    o3d.io.write_azure_kinect_mkv_metadata('{}/{}_intrinsic.json'.format(abspath, file_name), metadata)

    # Open the file and load the JSON
    with open(abspath + "/" + file_name + "_intrinsic.json") as f:
        data = json.load(f)
    
    height = data['height']
    width = data['width']
    intrinsics = data["intrinsic_matrix"]
    time_stamp = data["stream_length_usec"]

    camera_intrinsics = o3d.camera.PinholeCameraIntrinsic()
    cx = intrinsics[6]
    cy = intrinsics[7]
    fx = intrinsics[0]
    fy = intrinsics[4]
    camera_intrinsics.set_intrinsics(width, height, fx, fy, cx, cy)
    K = np.array([width,height,fx,fy,cx,cy])
    last_frame = None
    while not reader.is_eof():
        rgbda = reader.next_frame()
        if rgbda is None:
            continue
        last_frame = rgbda

    if last_frame is not None:
        #print("************No valid frames found in the .mkv file.**********")
    
        print("Got last_frame")
        #return last_frame
    else:
        print("************No valid frames found in the .mkv file.**********")
        #return None
    
    reader.close()
    #print(last_frame)
    #print((np.array(last_frame.color)).dtype)
    #print((np.array(last_frame.depth)).dtype)
    return [np.array(last_frame.color), np.array(last_frame.depth), K]

def process_single_frame(single_frame):
    ws = 2
    flying_pixel_filter_threshold = 1000
    color_np = single_frame[0]
    depth_np = single_frame[1]
    K = single_frame[2]
    
    depth_np = pad_depth_to_color(depth_np, color_np)

    depth_o3d = o3d.geometry.Image(np.ascontiguousarray(depth_np))
    color_o3d = o3d.geometry.Image(np.ascontiguousarray(color_np))

    rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
                color_o3d, depth_o3d, depth_trunc=4.0, convert_rgb_to_intensity=False)
    
    camera_intrinsics = o3d.camera.PinholeCameraIntrinsic()
    camera_intrinsics.set_intrinsics(int(K[0]),int(K[1]),K[2],K[3],K[4],K[5])

    volume = o3d.pipelines.integration.ScalableTSDFVolume(
                    voxel_length= 4.0/ 512.0,
                    sdf_trunc=0.4,
                    color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)

    volume.integrate(
        rgbdc, 
        camera_intrinsics,
        np.eye(4),
    )
    pcd_tsdf = volume.extract_point_cloud()
    result_mask = numba_eliminate_flying_pixels(depth_np.copy(), ws, flying_pixel_filter_threshold)
    depth_np[result_mask > flying_pixel_filter_threshold] = 0
    # Re-insert filtered depth into the rgbd image
    rgbdc.depth = o3d.geometry.Image(depth_np)

    # Apply maskrcnn to filtered depth image
    masked_rgbd = segment_images_modified(rgbdc)
    # necessary lol
    rgbdc_new = o3d.geometry.RGBDImage.create_from_color_and_depth(
                    masked_rgbd.color, masked_rgbd.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
    pcd_tmp = o3d.geometry.PointCloud.create_from_rgbd_image(rgbdc_new,
                                                        camera_intrinsics)

    return pcd_tmp, pcd_tsdf

tform_path = '/home/vigir3d/Datasets/cattle_scans/farm_07_24/Animal_calib/'
tforms = load_calibration(tform_path,5)

files_raw = glob.glob(data_path+'/*.mkv')
abspath = data_path
files = filter_file_names(files_raw)
files.sort()
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Error)

NUM_PROCESSES = 4
s = time.time()


# Read the mkv
read1 = time.time()
with Pool(processes=NUM_PROCESSES) as pool:
    rgbd_np =  pool.map(process_file, range(len(files)))

read2 = time.time()




# Assuming rgbd_np is a list of frames
t1 = time.time()
pcds, pcds_tsdf = zip(*[process_single_frame(frame) for frame in rgbd_np])
t2 = time.time()

# registration
r1 = time.time()
pcd_all, ptsdf_all = perform_pairwise_alignment_par(ptsdf,pcds)
r2 = time.time()

p1 = time.time()
pcd_all.estimate_normals()
# cluster first to remove extra noise
pcd_clustered = cluster_point_cloud_new(pcd_all)
# After clustering, we upsample from the initial
p2 = time.time()

u1 = time.time()
pcd_upsampled = upsample_using_reference_normals_new(pcd_clustered, ptsdf_all)
u2 = time.time()

tn1 = time.time()
pcd_downsampled = pcd_upsampled.voxel_down_sample(0.01)
#pcd_downsampled.orient_normals_towards_camera_location()
pcd_downsampled.orient_normals_consistent_tangent_plane(30)
tn2 = time.time()

m1 = time.time()
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd_downsampled, depth=6)

sa = mesh.get_surface_area()
if(mesh.is_watertight()):
    #print("Is watertight 1")
    v =  mesh.get_volume()
else : 
    v = 0.0
m2 = time.time()
animal_id = os.path.basename(os.path.normpath(data_path))
save_results(animal_id, data_path, mesh, pcd_downsampled, sa, v)

e = time.time()

print("Total time: ", e-s, " s")


print("Read mkvs: ", read2-read1, " s")
print("Create and segment PCDS: ", t2-t1, " s")

print("Register: ", l2-l1, " s")

print("Preprocess: ", p2-p1, " s")

print("Upsampling: ", u2-u1, " s")
print("Normal orientation: ", tn2-tn1, " s")    
print("Meshing: ", m2-m1, " s")

In [ ]:
p = o3d.io.read_point_cloud("/home/vigir3d/Datasets/cattle_scans/farm_07_28/Animal_c1/c1_cleaned.ply")

In [ ]:
p.orient_normals_consistent_tangent_plane(30)

In [ ]:
o3d.io.write_point_cloud("/home/vigir3d/Datasets/cattle_scans/farm_07_28/Animal_c1/c1_cleaned_normals_oriented.ply",p)

In [ ]:
o3d.visualization.draw_geometries([p])

In [ ]:
pcd_demo_cow_path  = '/home/vigir3d/Datasets/cattle_scans/FirstOne/'
pcd_demo_cow = o3d.io.read_point_cloud(pcd_demo_cow_path+'registered_cow_cleaned.ply')

In [ ]:
pcd_demo_cow.estimate_normals()
pcd_demo_cow.orient_normals_consistent_tangent_plane(30)


In [ ]:
o3d.io.write_point_cloud(pcd_demo_cow_path+"registered_cow_corrected_normals.ply", pcd_demo_cow)

In [ ]:
o3d.visualization.draw_geometries([pcd_demo_cow])

In [ ]:
intrinsics[0]

In [ ]:
dpath = '/home/vigir3d/Datasets/cattle_scans/farm_scan1/Animal_482_2/'

In [ ]:

#dpath = '/home/vigir3d/Datasets/cattle_scans/farm_07_28/Animal_box3_u/'
depth = o3d.io.read_image(dpath + 'Animal_box3_u_nano_11_depth.png')
color = o3d.io.read_image(dpath + 'Animal_box3_u_nano_11.jpg')
K = np.loadtxt(dpath+"Animal_box3_u_nano_11_intrinsics.txt")

camera_intrinsics = o3d.camera.PinholeCameraIntrinsic()
camera_intrinsics.set_intrinsics(int(K[0]),int(K[1]),K[2],K[3],K[4],K[5])

rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color, depth, depth_trunc=4.0, convert_rgb_to_intensity=False)

volume = o3d.pipelines.integration.ScalableTSDFVolume(
            voxel_length= 4.0/ 512.0,
            sdf_trunc=0.04,
            color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)

volume.integrate(
    rgbdc,
    camera_intrinsics,
    np.eye(4),
)

In [ ]:
dpath = '/home/vigir3d/Datasets/cattle_scans/farm_scan1/Animal_482_2/'

# Remove the trailing slash if it exists



depth_image_array = np.asarray(last_frame.depth)
        rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        last_frame.color, last_frame.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
        volume = o3d.pipelines.integration.ScalableTSDFVolume(
                    voxel_length= 4.0/ 512.0,
                    sdf_trunc=0.4,
                    color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)

        volume.integrate(
            rgbdc, # rgbdc originally - trying with masked_rgbd
            camera_intrinsics,
            np.eye(4),
        )
#         pcd_raw = backproject_o3d(last_frame,camera_intrinsics)
        pcd_tsdf = volume.extract_point_cloud()
        # First eliminate flying pixels before integrating the depth map
        result_mask = numba_eliminate_flying_pixels(depth_image_array.copy(), ws, flying_pixel_filter_threshold)
        depth_image_array[result_mask > flying_pixel_filter_threshold] = 0
        # Re-insert filtered depth into the rgbd image
        last_frame.depth = o3d.geometry.Image(depth_image_array)
      
        # Apply maskrcnn to filtered depth image
        masked_rgbd = segment_images_modified(last_frame)
        # necessary lol
        rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        masked_rgbd.color, masked_rgbd.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
        pcd_tmp = o3d.geometry.PointCloud.create_from_rgbd_image(rgbdc,
                                                            camera_intrinsics)

# Latest Working Block

In [ ]:
dpath = '/home/vigir3d/Datasets/cattle_scans/farm_scan1/Animal_482_2/'
def process_file_list_comp(file_id, dpath):
    ws = 2
    flying_pixel_filter_threshold = 0.1
    # Format filenames
    dpath = dpath.rstrip('/')

    # Split by the directory separator and get the last part
    suffix = os.path.basename(dpath)
    
    file_suffix = suffix + "_nano_" + str(file_id)
    # Construct full file paths using os.path.join for better cross-platform compatibility
    depth_file = os.path.join(dpath, file_suffix + "_depth.png")
    color_file = os.path.join(dpath, file_suffix + ".jpg")
    intrinsics_file = os.path.join(dpath, file_suffix + "_intrinsics.txt")
    # Read the files
    depth = o3d.io.read_image(depth_file)
    color = o3d.io.read_image(color_file)
    #depth = o3d.io.read_image(dpath + 'Animal_box3_u_nano_11_depth.png')
    #color = o3d.io.read_image(dpath + 'Animal_box3_u_nano_11.jpg')
    K = np.loadtxt(intrinsics_file)
      

    camera_intrinsics = o3d.camera.PinholeCameraIntrinsic()
    camera_intrinsics.set_intrinsics(int(K[0]), int(K[1]), K[2], K[3], K[4], K[5])

    rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color, depth, depth_trunc=4.0, convert_rgb_to_intensity=False
    )

    volume = o3d.pipelines.integration.ScalableTSDFVolume(
        voxel_length=4.0 / 512.0, sdf_trunc=0.04, color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8
    )

    volume.integrate(rgbdc, camera_intrinsics, np.eye(4))
    pcd_tsdf = volume.extract_point_cloud()
    depth_np = np.asarray(rgbdc.depth)
    result_mask = numba_eliminate_flying_pixels(depth_np.copy(), ws, flying_pixel_filter_threshold)
    depth_np[result_mask > flying_pixel_filter_threshold] = 0
    # Re-insert filtered depth into the rgbd image
    rgbdc.depth = o3d.geometry.Image(depth_np)
    # Apply maskrcnn to filtered depth image
    masked_rgbd = segment_images_modified(rgbdc)
    # necessary lol
    rgbdc_new = o3d.geometry.RGBDImage.create_from_color_and_depth(
                    masked_rgbd.color, masked_rgbd.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
    pcd_tmp = o3d.geometry.PointCloud.create_from_rgbd_image(rgbdc_new,
                                                        camera_intrinsics)
    return pcd_tsdf, pcd_tmp 


if __name__ == "__main__":
    dpath = '/home/vigir3d/Datasets/cattle_scans/farm_scan1/Animal_482_2/'
    file_ids = [11,12,14,16,17,18]  # exclude head cameras

    # Generate a list of volumes using list comprehension
    start = time.time()
    results = [process_file_list_comp(file_id, dpath) for file_id in file_ids]
    ptsdf, pcds = zip(*results)
    l = False
    if l :
        pcd_all, ptsdf_all = perform_pairwise_alignment(ptsdf,pcds)
        pcd_all.estimate_normals()
        pcd_clustered = cluster_point_cloud_new(pcd_all)
        pcd_upsampled = upsample_using_reference_normals_new(pcd_clustered, ptsdf_all)
        pcd_downsampled = pcd_upsampled.voxel_down_sample(0.01)
        pcd_downsampled.orient_normals_consistent_tangent_plane(30)
        with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
            mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd_downsampled, depth=6)

        sa = mesh.get_surface_area()
        if(mesh.is_watertight()):
            #print("Is watertight 1")
            v =  mesh.get_volume()
        else : 
            v = 0.0
        animal_id = os.path.basename(os.path.normpath(data_path))
        save_results(animal_id, data_path, mesh, pcd_downsampled, sa, v)


    end = time.time()

    print("Duration was:", end-start, "s")

In [ ]:
for i in range(len(pcds)):
    o3d.visualization.draw_geometries([pcds[i]])

In [ ]:
s1 = time.time()
pcd_all, ptsdf_all = perform_pairwise_alignment(ptsdf,pcds)
pcd_all.estimate_normals()
pcd_clustered = cluster_point_cloud_new(pcd_all)

s2 = time.time()

print("duration: ", s2-s1, "s")

In [ ]:
for i in range(len(pcds)):
    o3d.visualization.draw_geometries([ptsdf[i]])

In [ ]:
def perform_pairwise_alignment(pcds_tsdf,pcds_cropped):
    #"""Compute and apply transformations."""
    t01 = colored_ICP(pcds_tsdf[0], pcds_tsdf[1])
    t52 = colored_ICP(pcds_tsdf[5], pcds_tsdf[2])
    t43 = colored_ICP(pcds_tsdf[4], pcds_tsdf[3])
    t12 = colored_ICP(pcds_tsdf[1], pcds_tsdf[2])
    t32 = colored_ICP(pcds_tsdf[3], pcds_tsdf[2])

    H0 = t12 @ t01
    H1 = t12
    H3 = t32
    H4 = t32 @ t43
    H5 = t52

    # Transform the point clouds
    p0 = copy.deepcopy(pcds_cropped[0]).transform(H0)
    p1 = copy.deepcopy(pcds_cropped[1]).transform(H1)
    p3 = copy.deepcopy(pcds_cropped[3]).transform(H3)
    p4 = copy.deepcopy(pcds_cropped[4]).transform(H4)
    p5 = copy.deepcopy(pcds_cropped[5]).transform(H5)
    
    d0 = copy.deepcopy(pcds_tsdf[0]).transform(H0)
    d1 = copy.deepcopy(pcds_tsdf[1]).transform(H1)
    d3 = copy.deepcopy(pcds_tsdf[3]).transform(H3)
    d4 = copy.deepcopy(pcds_tsdf[4]).transform(H4)
    d5 = copy.deepcopy(pcds_tsdf[5]).transform(H5)
    
    pcd_combined = o3d.geometry.PointCloud()
    pcd_combined = p0+p1+pcds_cropped[2]+p3+p4+p5
    
    ptsdf_combined = o3d.geometry.PointCloud()
    ptsdf_combined = d0+d1+pcds_tsdf[2]+d3+d4+d5

    return pcd_combined, ptsdf_combined

In [ ]:
# Assuming rgbd_np is a list of frames
t1 = time.time()
pcds, pcds_tsdf = zip(*[process_single_frame(frame) for frame in rgbd_np])
t2 = time.time()

# registration
r1 = time.time()
pcd_all, ptsdf_all = perform_pairwise_alignment_par(ptsdf,pcds)
r2 = time.time()

p1 = time.time()
pcd_all.estimate_normals()
# cluster first to remove extra noise
pcd_clustered = cluster_point_cloud_new(pcd_all)
# After clustering, we upsample from the initial
p2 = time.time()

u1 = time.time()
pcd_upsampled = upsample_using_reference_normals_new(pcd_clustered, ptsdf_all)
u2 = time.time()

tn1 = time.time()
pcd_downsampled = pcd_upsampled.voxel_down_sample(0.01)
#pcd_downsampled.orient_normals_towards_camera_location()
pcd_downsampled.orient_normals_consistent_tangent_plane(30)
tn2 = time.time()

m1 = time.time()
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd_downsampled, depth=6)

sa = mesh.get_surface_area()
if(mesh.is_watertight()):
    #print("Is watertight 1")
    v =  mesh.get_volume()
else : 
    v = 0.0
m2 = time.time()
animal_id = os.path.basename(os.path.normpath(data_path))
save_results(animal_id, data_path, mesh, pcd_downsampled, sa, v)

e = time.time()

print("Total time: ", e-s, " s")


print("Read mkvs: ", read2-read1, " s")
print("Create and segment PCDS: ", t2-t1, " s")

print("Register: ", l2-l1, " s")

print("Preprocess: ", p2-p1, " s")

print("Upsampling: ", u2-u1, " s")
print("Normal orientation: ", tn2-tn1, " s")    
print("Meshing: ", m2-m1, " s")

In [ ]:
o3d.visualization.draw_geometries([pcds[0]])

In [ ]:
pcds_tsdf

In [ ]:
 file_ids = range(11, 17) 

In [ ]:
from PIL import Image
p = segment_images_modified(rgbadc)

In [ ]:
def segment_images_modified(last_frame):
    depth_PIL = Image.fromarray(np.asarray(last_frame.depth)).convert("RGB")
    rgb_image_array = np.asarray(last_frame.color)
    depth_image_array = np.asarray(last_frame.depth)
    rgb_PIL = Image.fromarray(rgb_image_array)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    rgb_mask = get_model_mask(model_rgb, rgb_PIL)
    depth_mask = get_model_mask(model_depth, depth_PIL)

    if depth_mask.nelement() == 0:
        mask_combined = rgb_mask
    else:
        mask_combined = depth_mask | rgb_mask  # 1 vote arbitration (OR the masks)

    # Convert tensor to numpy array and ensure the right datatype

    mask_combined = mask_combined.numpy().astype(rgb_image_array.dtype)
    mask_image = mask_combined.swapaxes(0, 2).swapaxes(0, 1)
    mask_image = (mask_image > 0).astype(rgb_image_array.dtype)   

    fg_image_rgb = rgb_image_array * mask_image

    # For the depth image:
    squeezed_mask = np.squeeze(mask_image)
    fg_image_depth = depth_image_array * squeezed_mask
    
    last_frame.color = o3d.geometry.Image(fg_image_rgb)
    last_frame.depth = o3d.geometry.Image(fg_image_depth)

    return last_frame


In [ ]:
pcd = volume.extract_point_cloud()

In [ ]:
pcd

In [ ]:
o3d.visualization.draw_geometries([pcd])

In [ ]:
import concurrent.futures
#import numpy as np
#import open3d as o3d
from functools import partial


def wrapper_colored_ICP(args, function):
    return function(args[0], args[1], args[2], args[3])

def perform_pairwise_alignment_par(pcds_tsdf, pcds_cropped):
    """Compute and apply transformations."""

    # Extracting points and colors as numpy arrays
    points = [np.asarray(p.points) for p in pcds_tsdf]
    colors = [np.asarray(p.colors) for p in pcds_tsdf]

    # Passing tuples to the executor.map function
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # Use partial to pass colored_ICP as the function to be called by wrapper_colored_ICP
        wrapped_func = partial(wrapper_colored_ICP, function=colored_ICP_par)
        
        transformations = list(executor.map(wrapped_func, [(points[0], colors[0], points[1], colors[1]),
                                                           (points[5], colors[5], points[2], colors[2]),
                                                           (points[4], colors[4], points[3], colors[3]),
                                                           (points[1], colors[1], points[2], colors[2]),
                                                           (points[3], colors[3], points[2], colors[2])]))

    t01, t52, t43, t12, t32 = transformations

    H0 = np.dot(t12, t01)
    H1 = t12
    H3 = t32
    H4 = np.dot(t32, t43)
    H5 = t52

    # Transform the point clouds directly using Open3D
    p0 = pcds_cropped[0].transform(H0)
    p1 = pcds_cropped[1].transform(H1)
    p3 = pcds_cropped[3].transform(H3)
    p4 = pcds_cropped[4].transform(H4)
    p5 = pcds_cropped[5].transform(H5)
    
    d0 = pcds_tsdf[0].transform(H0)
    d1 = pcds_tsdf[1].transform(H1)
    d3 = pcds_tsdf[3].transform(H3)
    d4 = pcds_tsdf[4].transform(H4)
    d5 = pcds_tsdf[5].transform(H5)
    
    pcd_combined = p0 + p1 + pcds_cropped[2] + p3 + p4 + p5
    ptsdf_combined = d0 + d1 + pcds_tsdf[2] + d3 + d4 + d5

    return pcd_combined, ptsdf_combined


In [ ]:
import concurrent.futures
import copy
import numpy as np
import open3d as o3d

def colored_ICP_par(src_pts, src_c, tgt_pts, tgt_c):
    
    source = o3d.geometry.PointCloud()
    source.points = o3d.utility.Vector3dVector(src_pts)
    source.colors =  o3d.utility.Vector3dVector(src_c)
    
    target = o3d.geometry.PointCloud()

    target.points = o3d.utility.Vector3dVector(tgt_pts)
    target.colors =  o3d.utility.Vector3dVector(tgt_c)
    
    voxel_radius = [0.04, 0.02, 0.01]
    max_iter = [50, 30, 14]
    current_transformation = np.identity(4)
    #print("3. Colored point cloud registration")
    for scale in range(3):
        iters = max_iter[scale]
        radius = voxel_radius[scale]
        #print("iteration: ", iters, radius, scale)

        #print("3-1. Downsample with a voxel size %.2f" % radius)
        source_down = source.voxel_down_sample(radius)
        target_down = target.voxel_down_sample(radius)

        #print("3-2. Estimate normal.")
        source_down.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
        target_down.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))

        #print("3-3. Applying colored point cloud registration")
        result_icp = o3d.pipelines.registration.registration_colored_icp(
            source_down, target_down, radius, current_transformation,
            o3d.pipelines.registration.TransformationEstimationForColoredICP(),
            o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                              relative_rmse=1e-6,
                                                              max_iteration=iters))
        current_transformation = np.array(result_icp.transformation)
        
        return current_transformation



In [ ]:
def perform_pairwise_alignment_par(pcds_tsdf, pcds_cropped):
    """Compute and apply transformations."""

    # Extracting points and colors as numpy arrays
    points = [np.asarray(p.points) for p in pcds_tsdf]
    colors = [np.asarray(p.colors) for p in pcds_tsdf]

    # Defining a helper function to unpack and call colored_ICP
    def wrapper(args):
        return colored_ICP(args[0], args[1], args[2], args[3])

    # Passing tuples to the executor.map function
    with concurrent.futures.ProcessPoolExecutor() as executor:
        transformations = list(executor.map(wrapper, [(points[0], colors[0], points[1], colors[1]),
                                                      (points[5], colors[5], points[2], colors[2]),
                                                      (points[4], colors[4], points[3], colors[3]),
                                                      (points[1], colors[1], points[2], colors[2]),
                                                      (points[3], colors[3], points[2], colors[2])]))

    t01, t52, t43, t12, t32 = transformations

    H0 = np.dot(t12, t01)
    H1 = t12
    H3 = t32
    H4 = np.dot(t32, t43)
    H5 = t52

    # Transform the point clouds directly using Open3D
    p0 = pcds_cropped[0].transform(H0)
    p1 = pcds_cropped[1].transform(H1)
    p3 = pcds_cropped[3].transform(H3)
    p4 = pcds_cropped[4].transform(H4)
    p5 = pcds_cropped[5].transform(H5)
    
    d0 = pcds_tsdf[0].transform(H0)
    d1 = pcds_tsdf[1].transform(H1)
    d3 = pcds_tsdf[3].transform(H3)
    d4 = pcds_tsdf[4].transform(H4)
    d5 = pcds_tsdf[5].transform(H5)
    
    pcd_combined = p0 + p1 + pcds_cropped[2] + p3 + p4 + p5
    ptsdf_combined = d0 + d1 + pcds_tsdf[2] + d3 + d4 + d5

    return pcd_combined, ptsdf_combined


In [ ]:
print(rgbd_frames_glob[0])

In [ ]:
def cluster_point_cloud_new(outlier_cloud):
    cloud_colors = copy.deepcopy(np.asarray(outlier_cloud.colors).T)
    
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        labels = np.array(outlier_cloud.cluster_dbscan(eps=0.1, min_points=10, print_progress=False))

    # Identify the largest cluster
    values, counts = np.unique(labels, return_counts=True)
    ind = np.argmax(counts)
    largest_cluster_label = values[ind]
    #print(f"Largest cluster label: {largest_cluster_label}")

    # Filter points, normals, and colors for the largest cluster
    cloud_xyz = pcd2xyz(outlier_cloud)
    cloud_normals = pcd2normals(outlier_cloud)

    cloud_filtered = cloud_xyz[:, labels == largest_cluster_label]
    normals_filtered = cloud_normals[:, labels == largest_cluster_label]
    colors_filtered = cloud_colors[:, labels == largest_cluster_label]

    # Create a point cloud for the largest cluster
    pcd_filtered_largest_cluster = o3d.geometry.PointCloud()
    pcd_filtered_largest_cluster.points = o3d.utility.Vector3dVector(cloud_filtered.T)
    pcd_filtered_largest_cluster.normals = o3d.utility.Vector3dVector(normals_filtered.T)
    pcd_filtered_largest_cluster.colors = o3d.utility.Vector3dVector(colors_filtered.T)

    #o3d.visualization.draw_geometries([pcd_filtered_largest_cluster])
    return pcd_filtered_largest_cluster

In [ ]:

print(pp.is_cuda)


In [ ]:
o3d.visualization.draw_geometries([pp.to_legacy()])

In [ ]:
pp

In [ ]:
pp.orient_normals_consistent_tangent_plane(30)

In [ ]:
t1 = time.time()
pp.orient_normals_consistent_tangent_plane(30)
t2 = time.time()

t3 = time.time()
pcd_all.orient_normals_consistent_tangent_plane(30)
t4 = time.time()

print("Tensor: ", t2-t1, " s | legacy: ", t4-t3, " s" )


In [ ]:
def cluster_point_cloud_tensor(pcd) :
    colors = pcd.point.colors
    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Debug) as cm:
        labels = pcd.cluster_dbscan(eps=0.02, min_points=10, print_progress=False)
        
    
    labels = labels.cpu()
    # Identify the largest cluster
    values, counts = np.unique(labels, return_counts=True)
    ind = np.argmax(counts)
    largest_cluster_label = values[ind]
    #print(f"Largest cluster label: {largest_cluster_label}")

    colors[labels < 0] = 0
    pcd.point.colors = colors
    return pcd
#     o3d.visualization.draw_geometries([pcd.to_legacy()],
#                                       zoom=0.455,
#                                       front=[-0.4999, -0.1659, -0.8499],
#                                       lookat=[2.1813, 2.0619, 2.0999],
#                                       up=k[0.1204, -0.9852, 0.1215])


In [ ]:
def upsample_using_reference_normals_t(sparse_pcd_t, dense_pcd_t, search_radius=0.02, angle_threshold=30):
    #device = o3d.core.Device("CUDA:0")  # Assuming you have a CUDA GPU. Adjust as needed.

    # Convert point clouds to tensor-based format and move to GPU
    #sparse_pcd_t = o3d.t.geometry.PointCloud.from_legacy(sparse_pcd).to(device)
    #dense_pcd_t = o3d.t.geometry.PointCloud.from_legacy(dense_pcd).to(device)

    # Ensure point clouds have normals
    #if sparse_pcd_t.point.normals is None:
    sparse_pcd_t.estimate_normals()
    #if dense_pcd_t.point.normals is None:
    dense_pcd_t.estimate_normals()

    kdtree_t = o3d.t.geometry.KDTreeFlann(dense_pcd_t)

    upsampled_points = sparse_pcd_t.points.clone()

    for i, point in enumerate(sparse_pcd_t.points):
        [k, idx, _] = kdtree_t.search_radius_vector_3d(point, search_radius)
        
        neighbors = dense_pcd_t.points[idx]
        neighbor_normals = dense_pcd_t.normals[idx]

        # Calculate angles
        dot_products = sparse_pcd_t.normals[i].dot(neighbor_normals.T)
        angles = dot_products.acos().to(o3d.core.Dtype.Float32).asin() * (180.0 / np.pi)

        # Filtering neighbors
        valid_neighbors = neighbors[angles < angle_threshold]

        upsampled_points = upsampled_points.append(valid_neighbors)

    upsampled_pcd_t = o3d.t.geometry.PointCloud(device)
    upsampled_pcd_t.point.positions = upsampled_points
    upsampled_pcd_t.estimate_normals()

    # Convert back to legacy format if needed
    upsampled_pcd = upsampled_pcd_t.to_legacy()

    return upsampled_pcd

In [ ]:
def cluster_point_cloud_tensor(pcd) :
    # Clustering the points
    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Debug) as cm:
        labels = pcd.cluster_dbscan(eps=0.02, min_points=10, print_progress=False)
    
    labels = labels.cpu().numpy()
    
    # Identify the largest cluster
    values, counts = np.unique(labels, return_counts=True)
    ind = np.argmax(counts)
    largest_cluster_label = values[ind]

    # Filter points belonging to the largest cluster
    largest_cluster_points = pcd.point.positions[labels == largest_cluster_label]
    

    # Create a new point cloud tensor for the largest cluster
    largest_cluster_pcd = o3d.t.geometry.PointCloud(largest_cluster_points)
    largest_cluster_pcd.point.colors = pcd.point.colors[labels == largest_cluster_label]     
    return largest_cluster_pcd


In [ ]:
pcd_t.point.positions

In [ ]:
import numpy as np
from scipy.spatial import cKDTree


def upsample_using_reference_normals_new(sparse_pcd, dense_pcd, search_radius=0.02, angle_threshold=30):
    # Ensure the point clouds have normals
    if not sparse_pcd.has_normals():
        sparse_pcd.estimate_normals()
    if not dense_pcd.has_normals():
        dense_pcd.estimate_normals()

    dense_points = np.asarray(dense_pcd.points)
    dense_tree = cKDTree(dense_points)

    sparse_points = np.asarray(sparse_pcd.points)
    sparse_normals = np.asarray(sparse_pcd.normals)
    
    # This retrieves the indices of all neighbors within the search_radius
    neighbor_indices = dense_tree.query_ball_point(sparse_points, search_radius, workers=-1)
    
    valid_indices = []
    for i, idx_row in enumerate(neighbor_indices):
        neighbors = dense_points[idx_row]
        neighbor_normals = np.asarray(dense_pcd.normals)[idx_row]

        # Calculate angles between sparse point's normal and all its neighbors' normals
        angles = np.degrees(np.arccos(np.clip(np.dot(sparse_normals[i], neighbor_normals.T), -1.0, 1.0)))

        # Filtering neighbors based on angle threshold
        valid_neighbor_indices = np.array(idx_row)[angles < angle_threshold]
        valid_indices.extend(valid_neighbor_indices)
#     valid_indices = []
#     for i, idx_row in enumerate(neighbor_indices):
#         neighbors = dense_points[idx_row]
#         neighbor_normals = np.asarray(dense_pcd.normals)[idx_row]
        
#         # Calculate angles between sparse point's normal and all its neighbors' normals
#         angles = np.degrees(np.arccos(np.clip(np.dot(sparse_normals[i], neighbor_normals.T), -1.0, 1.0)))

#         # Filtering neighbors based on angle threshold
#         valid_for_current = idx_row[angles < angle_threshold]
#         valid_indices.extend(valid_for_current)

    unique_valid_indices = np.unique(valid_indices)
    final_valid_neighbors = dense_points[unique_valid_indices]
    
    upsampled_points = np.vstack([sparse_points, final_valid_neighbors])

    upsampled_pcd = o3d.geometry.PointCloud()
    upsampled_pcd.points = o3d.utility.Vector3dVector(upsampled_points)
    upsampled_pcd.estimate_normals()

    return upsampled_pcd


def upsample_using_reference_normals(sparse_pcd, dense_pcd, search_radius=0.02, angle_threshold=30):
    """
    Efficiently upsample the sparse point cloud using the dense point cloud as reference.
    Considers the normals to ensure added points are consistent with the sparse cloud.

    Parameters:
    - sparse_pcd: The sparse point cloud
    - dense_pcd: The dense reference point cloud
    - search_radius: Radius to search for neighbors
    - angle_threshold: Maximum angle in degrees between normals to consider a point

    Returns:
    - A new upsampled point cloud
    """
    # Ensure the point clouds have normals
    if not sparse_pcd.has_normals():
        sparse_pcd.estimate_normals()
    if not dense_pcd.has_normals():
        dense_pcd.estimate_normals()

    kdtree = o3d.geometry.KDTreeFlann(dense_pcd)

    sparse_points = np.asarray(sparse_pcd.points)
    sparse_normals = np.asarray(sparse_pcd.normals)
    added_points = set(map(tuple, sparse_points))

    upsampled_points = list(sparse_points)

    for i, point in enumerate(sparse_points):
        [k, idx, _] = kdtree.search_radius_vector_3d(point, search_radius)
        
        neighbors = np.asarray(dense_pcd.points)[idx]
        neighbor_normals = np.asarray(dense_pcd.normals)[idx]

        # Calculate angles between sparse point's normal and all its neighbors' normals
        angles = np.degrees(np.arccos(np.clip(np.dot(sparse_normals[i], neighbor_normals.T), -1.0, 1.0)))

        # Filtering neighbors based on angle threshold
        valid_neighbors = neighbors[angles < angle_threshold]

        # Filtering out points that are already in the sparse cloud or have been added before
        unique_valid_neighbors = [tuple(neighbor) for neighbor in valid_neighbors if tuple(neighbor) not in added_points]

        upsampled_points.extend(unique_valid_neighbors)
        added_points.update(unique_valid_neighbors)

    upsampled_pcd = o3d.geometry.PointCloud()
    upsampled_pcd.points = o3d.utility.Vector3dVector(upsampled_points)
    upsampled_pcd.estimate_normals()

    return upsampled_pcd



In [ ]:
o3d.visuablization.draw_geometries([pp2])

In [ ]:
pp1

In [ ]:
pp2

In [ ]:
o3d.visualization.draw_geometries([pp1])

In [ ]:
o3d.visualization.draw_geometries([pcd_all])

In [ ]:
o3d.visualization.draw_geometries([ptsdf_all])

In [ ]:

t1 = time.time()
#pp1  = upsample_using_reference_normals(pcd_all, ptsdf_all)
t2 = time.time()

t3 = time.time()
pp2  = upsample_using_reference_normals_new(pcd_all, ptsdf_all)

t4 = time.time()


print("orig: ", t2-t1, " s | new: ", t4-t3, " s" )

In [ ]:
t1 = time.time()
pcd_clustered = cluster_point_cloud_new(pcd_all)
t2 = time.time()

t3 = time.time()
o3d_device = o3d.core.Device("CUDA:0") 
pcd_t = o3d.t.geometry.PointCloud.from_legacy(pcd_all).to(o3d_device)
pcd_t_tsdf = o3d.t.geometry.PointCloud.from_legacy(ptsdf_all).to(o3d_device)
pt = cluster_point_cloud_tensor(pcd_t)
p_up_t = upsample_using_reference_normals_t(pt,pcd_t_tsdf)
#pt.estimate_normals()
#pt.orient_normals_consistent_tangent_plane(30)
#pcd_cpu = pt.cpu()
#o3d.visualization.draw_geometries([pcd_cpu.to_legacy()])
t4 = time.time()
print("legacy: ", t2-t1, " s | Tensor: ", t4-t3, " s" )


In [ ]:
jit(nopython=True)
# def compute_neighbors_and_angles(sparse_normal, neighbors, neighbor_normals, added_points, angle_threshold):
#     angles = np.degrees(np.arccos(np.clip(np.dot(sparse_normal, neighbor_normals.T), -1.0, 1.0)))
#     valid_neighbors = neighbors[angles < angle_threshold]
#     unique_valid_neighbors = [neighbor for neighbor in valid_neighbors if tuple(neighbor) not in added_points]
#     return unique_valid_neighbors

@jit(nopython=True)
def compute_neighbors_and_angles(sparse_normal, neighbors, neighbor_normals, added_points, angle_threshold):
    angles = np.degrees(np.arccos(np.clip(np.dot(sparse_normal, neighbor_normals.T), -1.0, 1.0)))
    valid_neighbors = neighbors[angles < angle_threshold]
    unique_valid_neighbors = [tuple(neighbor) for neighbor in valid_neighbors if tuple(neighbor) not in added_points]
    return unique_valid_neighbors

def upsample_using_reference_normals_new(sparse_pcd, dense_pcd, search_radius=0.02, angle_threshold=30):
    # Ensure the point clouds have normals
    if not sparse_pcd.has_normals():
        sparse_pcd.estimate_normals()
    if not dense_pcd.has_normals():
        dense_pcd.estimate_normals()

    kdtree = o3d.geometry.KDTreeFlann(dense_pcd)

    sparse_points = np.asarray(sparse_pcd.points)
    sparse_normals = np.asarray(sparse_pcd.normals)
    added_points = set(map(tuple, sparse_points))

    upsampled_points = list(sparse_points)

    for i, point in enumerate(sparse_points):
        [k, idx, _] = kdtree.search_radius_vector_3d(point, search_radius)
        neighbors = np.asarray(dense_pcd.points)[idx]
        neighbor_normals = np.asarray(dense_pcd.normals)[idx]

        unique_valid_neighbors = compute_neighbors_and_angles(
            sparse_normals[i], neighbors, neighbor_normals, added_points, angle_threshold
        )

        upsampled_points.extend(unique_valid_neighbors)
        added_points.update(unique_valid_neighbors)

    upsampled_pcd = o3d.geometry.PointCloud()
    upsampled_pcd.points = o3d.utility.Vector3dVector(upsampled_points)
    upsampled_pcd.estimate_normals()

    return upsampled_pcd

In [ ]:
pcds, ptsdf = load_filter_pcds(data_path,tforms)
pcd_all, ptsdf_all = perform_pairwise_alignment(ptsdf,pcds)
pcd_all.estimate_normals()
# cluster first to remove extra noise
pcd_clustered = cluster_point_cloud_new(pcd_all)
# After clustering, we upsample from the initial
pcd_upsampled = upsample_using_reference_normals(pcd_clustered, ptsdf_all)
pcd_downsampled = pcd_upsampled.voxel_down_sample(0.01)
pcd_downsampled.orient_normals_consistent_tangent_plane(30)

In [ ]:
def upsample_using_reference_normals_new(sparse_pcd, dense_pcd, search_radius=0.02, angle_threshold=30):
    # Ensure the point clouds have normals
    if not sparse_pcd.has_normals():
        sparse_pcd.estimate_normals()
    if not dense_pcd.has_normals():
        dense_pcd.estimate_normals()

    kdtree = o3d.geometry.KDTreeFlann(dense_pcd)

    sparse_points = np.asarray(sparse_pcd.points)
    sparse_normals = np.asarray(sparse_pcd.normals)
    added_points = set(map(tuple, sparse_points))

    upsampled_points = list(sparse_points)

    for i, point in enumerate(sparse_points):
        [k, idx, _] = kdtree.search_radius_vector_3d(point, search_radius)
        neighbors = np.asarray(dense_pcd.points)[idx]
        neighbor_normals = np.asarray(dense_pcd.normals)[idx]

        unique_valid_neighbors = compute_neighbors_and_angles(
            sparse_normals[i], neighbors, neighbor_normals, added_points, angle_threshold
        )

        upsampled_points.extend(unique_valid_neighbors)
        added_points.update(unique_valid_neighbors)

    upsampled_pcd = o3d.geometry.PointCloud()
    upsampled_pcd.points = o3d.utility.Vector3dVector(upsampled_points)
    upsampled_pcd.estimate_normals()

    return upsampled_pcd

In [ ]:
r12,t12  = register_two_views_teaser(ptsdf[1],ptsdf[2],0.035)

In [ ]:
t01 = demo_manual_registration(ptsdf[0], ptsdf[1])

In [ ]:
for i in np.arange(0.02, 0.1, 0.005):
    print(i)
    r01,t01  = register_two_views_teaser(ptsdf[0],ptsdf[1],i)

In [ ]:
# 4 to 5 works
r45,t45  = register_two_views_teaser(ptsdf[4],ptsdf[5],0.035)

In [ ]:
# 3 to 5 works
r35,t35  = register_two_views_teaser(ptsdf[3],ptsdf[5],0.04)

In [ ]:
r52,t52  = register_two_views_teaser(ptsdf[5],ptsdf[2],0.06)

In [ ]:
pcd_all, ptsdf_all = perform_pairwise_alignment(ptsdf,pcds)


In [ ]:
for i in range(len(ptsdf)):
    o3d.visualization.draw_geometries([ptsdf[i]])

In [ ]:
for i in range(len(pcds)):
    o3d.visualization.draw_geometries([pcds[i]])

In [ ]:
o3d.visualization.draw_geometries(pcds)

In [ ]:
o3d.visualization.draw_geometries(ptsdf)

In [ ]:
d0 = t02
d1 = h12[1]
d3 = r32
d4 = r32@t43
d5 = r52

dt0 = copy.deepcopy(ptsdf[0]).transform(d0)
dt1 = copy.deepcopy(ptsdf[1]).transform(d1)
dt3 = copy.deepcopy(ptsdf[3]).transform(d3)
dt4 = copy.deepcopy(ptsdf[4]).transform(d4)
dt5 = copy.deepcopy(ptsdf[5]).transform(d5)
o3d.visualization.draw_geometries([dt0,dt1,dt3,dt4,dt5,ptsdf[2]])
pcd_comb = o3d.geometry.PointCloud()
pcd_comb = dt0+dt1+dt3+dt4+dt5+ptsdf[2]


In [ ]:
o3d.visualization.draw_geometries([pcd_comb])

In [ ]:
pdt0 = copy.deepcopy(pcds[0]).transform(d0)
pdt1 = copy.deepcopy(pcds[1]).transform(d1)
pdt3 = copy.deepcopy(pcds[3]).transform(d3)
pdt4 = copy.deepcopy(pcds[4]).transform(d4)
pdt5 = copy.deepcopy(pcds[5]).transform(d5)

In [ ]:
dpcd_comb = o3d.geometry.PointCloud()
dpcd_comb = pdt0+pdt1+pdt3+pdt4+pdt5+pcds[2]
o3d.visualization.draw_geometries([dpcd_comb])

In [ ]:
dpcd_comb.estimate_normals()
dpcd_comb.orient_normals_consistent_tangent_plane(30)

In [ ]:
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        meshd, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(dpcd_comb, depth=6)

o3d.visualization.draw_geometries([meshd])
sa1 = meshd.get_surface_area()
if(meshd.is_watertight()):
    print("Is watertight 1")
    v1 =  meshd.get_volume()
print(sa1)

In [ ]:
t02 = demo_manual_registration(ptsdf[0],pcd_comb)

In [ ]:
for i in np.arange(0.05,0.08,0.001):
    t02,r02 = register_two_views_teaser(ptsdf[0],pcd_comb,i)

In [ ]:
t01 = register_tw

In [ ]:
for i in np.arange(0.02,0.1,0.001):
    t01,r01  = register_two_views_teaser(ptsdf[0],ptsdf[3],i)

In [ ]:
t52, r52 = register_two_views_teaser(ptsdf[5],ptsdf[2],0.03)

In [ ]:
t43 = demo_manual_registration(ptsdf[4],ptsdf[3])

In [ ]:
t45,r45 = register_two_views_teaser(ptsdf[4],ptsdf[5],0.1)

In [ ]:
def perturb_transformation(T, rotation_angle_range=0.05, translation_range=0.01):
    # Extract rotation and translation
    R = T[:3, :3]
    t = T[:3, 3]
    
    # Perturb rotation
    axis = np.random.randn(3)
    axis /= np.linalg.norm(axis)
    angle = np.random.uniform(-rotation_angle_range, rotation_angle_range)
    small_rotation = o3d.geometry.get_rotation_matrix_from_axis_angle(axis * angle)
    R_perturbed = np.dot(R, small_rotation)
    
    # Perturb translation
    t_perturbed = t + np.random.uniform(-translation_range, translation_range, size=3)
    
    # Create perturbed transformation
    T_perturbed = np.eye(4)
    T_perturbed[:3, :3] = R_perturbed
    T_perturbed[:3, 3] = t_perturbed
    
    return T_perturbed

# Example usage:
T_original = np.array([
    [0.866, -0.5, 0, 0.5],
    [0.5, 0.866, 0, 0.5],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])
T_perturbed = perturb_transformation(T_original)
print(T_perturbed)


In [ ]:
r12_m = perturb_transformation(r12,0.2,0.2)
draw_registration_result(ptsdf[1],ptsdf[2],r12_m)

p12 = copy.deepcopy(ptsdf[1]).transform(r12_m)

o3d.visualization.draw_geometries([p12,ptsdf[2]])
r12_c = colored_ICP(p12,ptsdf[2])
draw_registration_result(p12,ptsdf[2],r12_c)

In [ ]:
r32_m = perturb_transformation(r32,0.2,0.2)
draw_registration_result(ptsdf[3],ptsdf[2],r32_m)

p = copy.deepcopy(ptsdf[3]).transform(r32_m)

o3d.visualization.draw_geometries([p,ptsdf[2]])
r = colored_ICP(p,ptsdf[2])
draw_registration_result(p,ptsdf[2],r)

In [ ]:
#h43 = register_two_views_teaser(ptsdf[4],ptsdf[3],0.04)

draw_registration_result(ptsdf[4],ptsdf[3],h43[1])

p4tf = copy.deepcopy(ptsdf[4]).transform(h43[1])

o3d.visualization.draw_geometries([p4tf,ptsdf[3]])
r4_3 = colored_ICP(p4tf,ptsdf[3])
draw_registration_result(p4tf,ptsdf[3],r4_3)

In [ ]:
t32,r32 = register_two_views_teaser(ptsdf[3],ptsdf[2],0.035)

#t32 = demo_manual_registration(ptsdf[3],ptsdf[2])

In [ ]:
for i in np.arange(0.03,0.1,0.001):
    h52 = register_two_views_teaser(ptsdf[4],ptsdf[3],i)

In [ ]:
h43 = register_two_views_teaser(ptsdf[4],ptsdf[3],0.04)

In [ ]:
h12,r12 = register_two_views_teaser(ptsdf[1],ptsdf[2],0.04)

In [ ]:
t01 = demo_manual_registration(ptsdf[0],ptsdf[1])

In [ ]:
t12= demo_manual_registration(ptsdf[1],ptsdf[2])

In [ ]:
t52 = demo_manual_registration(ptsdf[5],ptsdf[2])

In [ ]:
t34,r34 = register_two_views_teaser(ptsdf[3],ptsdf[4],0.03)

In [ ]:
t45, r45 = register_two_views_teaser(ptsdf[4],ptsdf[5],0.05)

In [ ]:
tforms = 

In [ ]:
h0 = t12@t01
h1 = t12
h3 = t52@r45@r34
h4 = t52@r45
h5 = t52

In [ ]:
p0 = copy.deepcopy(ptsdf[0]).transform(h0)
p1 = copy.deepcopy(ptsdf[1]).transform(h1)
p3 = copy.deepcopy(ptsdf[3]).transform(h3)
p4 = copy.deepcopy(ptsdf[4]).transform(h4)
p5 = copy.deepcopy(ptsdf[5]).transform(h5)

In [ ]:
c0 = copy.deepcopy(pcds[0]).transform(h0)
c1 = copy.deepcopy(pcds[1]).transform(h1)
c3 = copy.deepcopy(pcds[3]).transform(h3)
c4 = copy.deepcopy(pcds[4]).transform(h4)
c5 = copy.deepcopy(pcds[5]).transform(h5)

In [ ]:

def register_two_views_teaser(A_pcd_raw,B_pcd_raw,VOXEL_SIZE):
    
    VISUALIZE = True
    A_pcd = A_pcd_raw.voxel_down_sample(voxel_size=VOXEL_SIZE)
    B_pcd = B_pcd_raw.voxel_down_sample(voxel_size=VOXEL_SIZE)
    #if VISUALIZE:
     #   o3d.visualization.draw_geometries([A_pcd,B_pcd]) # plot downsampled A and B 

    A_xyz = pcd2xyz(A_pcd) # np array of size 3 by N
    B_xyz = pcd2xyz(B_pcd) # np array of size 3 by M

    print("Extracting FPFH features")
    # extract FPFH features
    A_feats = extract_fpfh(A_pcd,VOXEL_SIZE)
    B_feats = extract_fpfh(B_pcd,VOXEL_SIZE)
    print(A_feats.shape)
    print("Computing FPFH correspondences")
    # establish correspondences by nearest neighbour search in feature space
    corrs_A, corrs_B = find_correspondences(
        A_feats, B_feats, mutual_filter=True)
    A_corr = A_xyz[:,corrs_A] # np array of size 3 by num_corrs
    B_corr = B_xyz[:,corrs_B] # np array of size 3 by num_corrs

    num_corrs = A_corr.shape[1]
    print(f'FPFH generates {num_corrs} putative correspondences.')

    # visualize the point clouds together with feature correspondenc
    # robust global registration using TEASER++
    NOISE_BOUND = VOXEL_SIZE
    teaser_solver = get_teaser_solver(NOISE_BOUND)
    teaser_solver.solve(A_corr,B_corr)
    solution = teaser_solver.getSolution()
    R_teaser = solution.rotation
    t_teaser = solution.translation
    T_teaser = Rt2T(R_teaser,t_teaser)

    # Visualize the registration results
    A_pcd_T_teaser = copy.deepcopy(A_pcd).transform(T_teaser)
    #o3d.visualization.draw_geometries([A_pcd_T_teaser,B_pcd])

    # local refinement using ICP
    icp_sol = o3d.pipelines.registration.registration_icp(
          A_pcd, B_pcd, NOISE_BOUND, T_teaser,
          o3d.pipelines.registration.TransformationEstimationPointToPoint(),
          o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=100))
    T_icp = icp_sol.transformation

    # visualize the registration after ICP refinement
    A_pcd_T_icp = copy.deepcopy(A_pcd).transform(T_icp)
    if VISUALIZE:
        Acopy = copy.deepcopy(A_pcd_T_icp).paint_uniform_color([0.0,0.0,1])
        Bcopy = copy.deepcopy(B_pcd).paint_uniform_color([1.0,0.0,0.0])
        o3d.visualization.draw_geometries([Acopy,Bcopy])
    tformed_A = copy.deepcopy(A_pcd_raw).transform(T_icp)
    res = o3d.geometry.PointCloud()
    res = tformed_A + B_pcd_raw
    
    return res,T_icp

def pcd2xyz(pcd):
    return np.asarray(pcd.points).T

def extract_fpfh(pcd, voxel_size):
    radius_normal = voxel_size * 2
    pcd.estimate_normals(
      o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    fpfh = o3d.pipelines.registration.compute_fpfh_feature(
      pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return np.array(fpfh.data).T

def find_knn_cpu(feat0, feat1, knn=1, return_distance=False):
    feat1tree = cKDTree(feat1)
    dists, nn_inds = feat1tree.query(feat0, k=knn, workers=10)
    if return_distance:
        return nn_inds, dists
    else:
        return nn_inds

def find_correspondences(feats0, feats1, mutual_filter=True):
    nns01 = find_knn_cpu(feats0, feats1, knn=1, return_distance=False)
    corres01_idx0 = np.arange(len(nns01))
    corres01_idx1 = nns01

    if not mutual_filter:
        return corres01_idx0, corres01_idx1

    nns10 = find_knn_cpu(feats1, feats0, knn=1, return_distance=False)
    corres10_idx1 = np.arange(len(nns10))
    corres10_idx0 = nns10

    mutual_filter = (corres10_idx0[corres01_idx1] == corres01_idx0)
    corres_idx0 = corres01_idx0[mutual_filter]
    corres_idx1 = corres01_idx1[mutual_filter]

    return corres_idx0, corres_idx1

def get_teaser_solver(noise_bound):
    solver_params = teaserpp_python.RobustRegistrationSolver.Params()
    solver_params.cbar2 = 1.0
    solver_params.noise_bound = noise_bound
    solver_params.estimate_scaling = False
    solver_params.inlier_selection_mode = \
        teaserpp_python.RobustRegistrationSolver.INLIER_SELECTION_MODE.PMC_EXACT
    solver_params.rotation_tim_graph = \
        teaserpp_python.RobustRegistrationSolver.INLIER_GRAPH_FORMULATION.CHAIN
    solver_params.rotation_estimation_algorithm = \
        teaserpp_python.RobustRegistrationSolver.ROTATION_ESTIMATION_ALGORITHM.GNC_TLS
    solver_params.rotation_gnc_factor = 1.4
    solver_params.rotation_max_iterations = 10000
    solver_params.rotation_cost_threshold = 1e-16
    solver = teaserpp_python.RobustRegistrationSolver(solver_params)
    return solver

def Rt2T(R,t):
    T = np.identity(4)
    T[:3,:3] = R
    T[:3,3] = t
    return T 


In [ ]:
o3d.visualization.draw_geometries([c0,c1,pcds[2],c3,c4,c5])

In [ ]:
pcds_all = o3d.geometry.PointCloud()
pcds_all = c0+c1+pcds[2]+c3+c4+c5

In [ ]:
o3d.visualization.draw_geometries([pcds_all])
o3d.visualization.draw_geometries([pcds_all_tsdf])

In [ ]:
o3d.visualization.draw_geometries([pcds_all])
o3d.io.write_point_cloud(spath+"pcd_animal_482_2_no_ds.ply",pcds_all)

In [ ]:
pcds_all_tsdf = o3d.geometry.PointCloud()
pcds_all_tsdf = p0+p1+ptsdf[2]+p3+p4+p5
o3d.io.write_point_cloud(spath+"pcd_animal_482_2_tsdf.ply",pcds_all_tsdf)

In [ ]:
o3d.io.write_point_cloud(spath+"pcd_animal_482_2_no_normals.ply",pcds_all)

In [ ]:
spath

In [ ]:

def upsample_using_reference(sparse_pcd, dense_pcd, search_radius=0.05):
    """
    Efficiently upsample the sparse point cloud using the dense point cloud as reference.

    Parameters:
    - sparse_pcd: The sparse point cloud
    - dense_pcd: The dense reference point cloud
    - search_radius: Radius to search for neighbors

    Returns:
    - A new upsampled point cloud
    """
    kdtree = o3d.geometry.KDTreeFlann(dense_pcd)

    sparse_points = np.asarray(sparse_pcd.points)
    added_points = set(map(tuple, sparse_points))

    upsampled_points = list(sparse_points)

    for point in sparse_points:
        [k, idx, _] = kdtree.search_radius_vector_3d(point, search_radius)
        
        # Convert indices to points
        neighbors = np.asarray(dense_pcd.points)[idx]
        
        # Filtering out points that are already in the sparse cloud or have been added before
        unique_neighbors = [tuple(neighbor) for neighbor in neighbors if tuple(neighbor) not in added_points]
        
        upsampled_points.extend(unique_neighbors)
        added_points.update(unique_neighbors)

    upsampled_pcd = o3d.geometry.PointCloud()
    upsampled_pcd.points = o3d.utility.Vector3dVector(upsampled_points)

    return upsampled_pcd

In [ ]:
o3d.visualization.draw_geometries([upcdn])

In [ ]:
o3d.visualization.draw_geometries([upcdn])

In [ ]:
upcdn

In [ ]:
upcd

In [ ]:
o3d.visualization.draw_geometries([pcds_all])
o3d.visualization.draw_geometries([pcds_all_tsdf])
o3d.visualization.draw_geometries([upcd])
o3d.visualization.draw_geometries([upcdn])

In [ ]:
s1 = time.time()
upcd = upsample_using_reference(pcds_all,pcds_all_tsdf,0.01)
upcd.estimate_normals()
upcd.orient_normals_consistent_tangent_plane(30)
#o3d.geometry.PointCloud.orient_normals_consistent_tangent_plane(upcd,30)
e1 = time.time()

s2 = time.time()
upcdn = upsample_using_reference_normals(pcds_all,pcds_all_tsdf,0.01, 30)
upcdn.orient_normals_consistent_tangent_plane(30)
#o3d.geometry.PointCloud.orient_normals_consistent_tangent_plane(upcdn,30)
e2 = time.time()

d1 = e1-s1
d2 = e2-s2

print("first: ", d1, "s")

print("second: ", d2, "s")

In [ ]:
o3d.io.write_point_cloud(spath+"upsampled_cloud.ply", upcd)
o3d.io.write_point_cloud(spath+"upsampled_normal_based_cloud.ply", upcdn)

In [ ]:
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        mesh1, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(upcd, depth=6)

o3d.visualization.draw_geometries([mesh1])
sa1 = mesh1.get_surface_area()
if(mesh1.is_watertight()):
    print("Is watertight 1")
    v1 =  mesh1.get_volume()


with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        mesh2, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(upcdn, depth=6)

o3d.visualization.draw_geometries([mesh2])
sa2 = mesh2.get_surface_area()
if(mesh2.is_watertight()):
    print("Is watertight 2")
    v2 =  mesh2.get_volume()

with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        mesh3, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcds_all, depth=6)

o3d.visualization.draw_geometries([mesh3])
sa3 = mesh3.get_surface_area()
if(mesh3.is_watertight()):
    print("Is watertight 3")
    v3 =  mesh3.get_volume()

print("SA: ", sa1, ", ", sa2, ", ", sa3)

In [ ]:
#o3d.visualization.draw_geometries([upcdn])
o3d.geometry.PointCloud.orient_normals_consistent_tangent_plane(upcdn,30)
o3d.visualization.draw_geometries([upcdn])

In [ ]:
o3d.visualization.draw_geometries([pcds_all])

In [ ]:
pd = pcds_all.voxel_down_sample(voxel_sizbe=0.05)

In [ ]:
o3d.visualization.draw_geometries([pd])

In [ ]:
o3d.geometry.PointCloud.orient_normals_consistent_tangent_plane(pd,30)

In [ ]:
#o3d.geometry.PointCloud.orient_normals_consistent_tangent_plane(pcds_all,30)
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcds_all, depth=6)

o3d.visualization.draw_geometries([mesh])

In [ ]:
#upcd.estimate_normals()
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(upcdn, depth=6)

o3d.visualization.draw_geometries([mesh])

In [ ]:

def upsample_using_reference_normals(sparse_pcd, dense_pcd, search_radius=0.02, angle_threshold=30):
    """
    Efficiently upsample the sparse point cloud using the dense point cloud as reference.
    Considers the normals to ensure added points are consistent with the sparse cloud.

    Parameters:
    - sparse_pcd: The sparse point cloud
    - dense_pcd: The dense reference point cloud
    - search_radius: Radius to search for neighbors
    - angle_threshold: Maximum angle in degrees between normals to consider a point

    Returns:
    - A new upsampled point cloud
    """
    # Ensure the point clouds have normals
    if not sparse_pcd.has_normals():
        sparse_pcd.estimate_normals()
    if not dense_pcd.has_normals():
        dense_pcd.estimate_normals()

    kdtree = o3d.geometry.KDTreeFlann(dense_pcd)

    sparse_points = np.asarray(sparse_pcd.points)
    sparse_normals = np.asarray(sparse_pcd.normals)
    added_points = set(map(tuple, sparse_points))

    upsampled_points = list(sparse_points)

    for i, point in enumerate(sparse_points):
        [k, idx, _] = kdtree.search_radius_vector_3d(point, search_radius)
        
        neighbors = np.asarray(dense_pcd.points)[idx]
        neighbor_normals = np.asarray(dense_pcd.normals)[idx]

        # Calculate angles between sparse point's normal and all its neighbors' normals
        angles = np.degrees(np.arccos(np.clip(np.dot(sparse_normals[i], neighbor_normals.T), -1.0, 1.0)))

        # Filtering neighbors based on angle threshold
        valid_neighbors = neighbors[angles < angle_threshold]

        # Filtering out points that are already in the sparse cloud or have been added before
        unique_valid_neighbors = [tuple(neighbor) for neighbor in valid_neighbors if tuple(neighbor) not in added_points]

        upsampled_points.extend(unique_valid_neighbors)
        added_points.update(unique_valid_neighbors)

    upsampled_pcd = o3d.geometry.PointCloud()
    upsampled_pcd.points = o3d.utility.Vector3dVector(upsampled_points)
    upsampled_pcd.estimate_normals()

    return upsampled_pcd

# Sample usage
# sparse_cloud = o3d.io.read_point_cloud("path_to_sparse_point_cloud.ply")
# dense_cloud = o3d.io.read_point_cloud("path_to_dense_point_cloud.ply")
# upsampled_cloud = upsample_using_reference(sparse_cloud, dense_cloud)
# o3d.visualization.draw_geometries([upsampled_cloud])


In [ ]:
def upsample_using_reference(sparse_pcd, dense_pcd, search_radius=0.05):
    """
    Upsamples the sparse point cloud using the dense point cloud as reference.

    Parameters:
    - sparse_pcd: The sparse point cloud
    - dense_pcd: The dense reference point cloud
    - search_radius: Radius to search for neighbors

    Returns:
    - A new upsampled point cloud
    """
    # Construct KDTree for dense point cloud
    kdtree = o3d.geometry.KDTreeFlann(dense_pcd)

    # Convert sparse point cloud to numpy array
    sparse_points = np.asarray(sparse_pcd.points)

    # List to store upsampled points
    upsampled_points = list(sparse_points)

    # For each point in the sparse cloud
    for point in sparse_points:
        # Search for neighbors from the dense cloud within the search_radius
        [k, idx, _] = kdtree.search_radius_vector_3d(point, search_radius)

        # If we found any neighbors, add them to the upsampled list
        if k > 1:
            # Convert indices to points
            neighbors = np.asarray(dense_pcd.points)[idx]
            
            # Add neighbors to the upsampled list
            # We filter out points that are already in the sparse cloud
            for neighbor in neighbors:
                if not np.any(np.all(sparse_points == neighbor, axis=1)):
                    upsampled_points.append(neighbor)

    # Create a new point cloud for the upsampled points
    upsampled_pcd = o3d.geometry.PointCloud()
    upsampled_pcd.points = o3d.utility.Vector3dVector(upsampled_points)

    return upsampled_pcd

In [ ]:
pcds_all.estimate_normals()

In [ ]:
o3d.visualization.draw_geometries([pcd])

In [ ]:
spath = '/home/vigir3d/Datasets/cattle_scans/farm_scan1/Animal_482_2/'

In [ ]:
o3d.io.write_point_cloud(spath+"pcd_animal_482_2.ply",pcd)

In [ ]:
pcd_raw = o3d.io.read_point_cloud(spath+"pcd_animal_482_2.ply")

In [ ]:
pcd_raw2 = o3d.io.read_point_cloud(spath+"pcd_animal_482_2_no_normals.ply")

In [ ]:
pcd_raw.orient_normals_consistent_tangent_plane(30)

In [ ]:
o3d.visualization.draw_geometries([pcd_raw2])

In [ ]:
knn = o3d.geometry.KDTreeSearchParamKNN(30)

In [ ]:
pcd_raw2.estimate_normals(knn)

In [ ]:
o3d.visualization.draw_geometries([pcd_raw2])

In [ ]:
pcd_raw2.orient_normals_consistent_tangent_plane(30)

In [ ]:

#pcd = pcds_all.voxel_down_sample(voxel_size=0.05)
#o3d.visualization.draw_geometries([pcd])
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd_raw, depth=6)

o3d.visualization.draw_geometries([mesh])

sa = mesh.get_surface_area()
print("Downsampled: ", sa)

In [ ]:

#pcd = pcds_all.voxel_down_sample(voxel_size=0.05)
#o3d.visuaalization.draw_geometries([pcd])
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd_raw2, depth=6)

o3d.visualization.draw_geometries([mesh])

sa = mesh.get_surface_area()
print(sa)

In [ ]:
p_cl_1 = cluster_point_cloud_new(pcd_raw)
p_cl_2 = cluster_point_cloud_new(pcd_raw2)

In [ ]:
p_cl_2.voxel_down_sample(voxel_size=0.01)
o3d.visualization.draw_geometries([p_cl_2])

In [ ]:
p2  = p_cl_2.voxel_down_sample(voxel_size=0.01)
p3  = p_cl_2.voxel_down_sample(voxel_size=0.02)
p4  = p_cl_2.voxel_down_sample(voxel_size=0.03)


o3d.visualization.draw_geometries([p2])


In [ ]:
p_cl_2


In [ ]:
p2

In [ ]:
p3

In [ ]:
p4

In [ ]:
p2  = p_cl_2.voxel_down_sample(voxel_size=0.08)
o3d.visualization.draw_geometries([p_cl_2])
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        mesh1, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(p_cl_2, depth=6)

#o3d.visualization.draw_geometries([mesh])

sa1 = mesh1.get_surface_area()
print("Original:", sa1)


with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        mesh2, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(p2, depth=6)

#o3d.visualization.draw_geometries([mesh])

sa2 = mesh2.get_surface_area()
print("Downsampled: ", sa2)

In [ ]:
pcd_cl = cluster_point_cloud_new(pcd)

In [ ]:
o3d.visualization.draw_geometries([pcd_cl])

In [ ]:
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd_cl, depth=6)

o3d.visualization.draw_geometries([mesh])

In [ ]:
def cluster_point_cloud(outlier_cloud) :
    cloud_colors = np.asarray(outlier_cloud.colors).T
    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Error) as cm:
        labels = np.array(
            outlier_cloud.cluster_dbscan(eps=0.1, min_points=10, print_progress=True))

    max_label = labels.max()
    print(f"point cloud has {max_label + 1} clusters")
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    copy_outlier_cloud = copy.deepcopy(outlier_cloud)
    copy_outlier_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([copy_outlier_cloud])

    values, counts = np.unique(labels, return_counts=True)
    ind = np.argmax(counts)
    print(values[ind])  # prints the most frequent element

    cloud_xyz = pcd2xyz(outlier_cloud)
    cloud_normals  = pcd2normals(outlier_cloud)
    
    print("C: ", cloud_colors.shape, " N", cloud_normals.shape)
    cloud_filtered = cloud_xyz[:,(labels == 0)]
    normals_filtered = (cloud_normals[:,(labels == 0)])
    colors_filtered = (cloud_colors[:,(labels == 0)])
    pcd_filtered_largest_cluster = o3d.geometry.PointCloud()
    pcd_filtered_largest_cluster.points = o3d.utility.Vector3dVector(cloud_filtered.T)
    pcd_filtered_largest_cluster.normals = o3d.utility.Vector3dVector(normals_filtered.T)
    pcd_filtered_largest_cluster.colors = o3d.utility.Vector3dVector(colors_filtered.T)
    # copy the normals also
    o3d.visualization.draw_geometries([pcd_filtered_largest_cluster])
    return pcd_filtered_largest_cluster


def pcd2normals(pcd):
    return np.asarray(pcd.normals).T

In [ ]:
tform_path = '/home/vigir3d/Datasets/cattle_scans/farm_scan1/Animal_482_1/'
h0 = np.loadtxt(tforma_path+"htm_0_3.txt")

In [ ]:
o3d.visualization.draw_geometries([tforms_apriltag[0]])

In [ ]:
o3d.visualization.draw_geometries([pcds[0]])

In [ ]:
o3d.visualization.draw_geometries([pt[0]])

In [ ]:
o3d.visualization.draw_geometries([p[0]])

In [ ]:
draw_registration_result(pt[0],pt[0], h0)

In [ ]:
o3d.visualization.draw_geometries(pt)

In [ ]:
data_path = '/home/vigir3d/Datasets/cattle_scans/farm_scan1/Animal_482_2'

ws = 2
flying_pixel_filter_threshold = 10000



reader = o3d.io.AzureKinectMKVReader()
abspath = data_path
pcds = []
filtered_pcds = []


files_raw = glob.glob(data_path+'/*.mkv')
files = filter_file_names(files_raw)
files.sort()
pcds_raw = []
pcds_tsdf = []

list_size = len(files)
rgbd_frames = [None] * list_size

for i in range(len(files)): # for each view

    inFile = files[i]
    fname = inFile.split('/')[-1]
    file_name = fname.split('.mkv')[0]
    reader.open(inFile)
    if not reader.is_opened():
        raise RuntimeError("Unable to open file {}".format(inFile))
    metadata = reader.get_metadata()

    # write the metadata to a JSON file since that seems to be the only
    # way to retrieve that data
    o3d.io.write_azure_kinect_mkv_metadata(
                '{}/{}_intrinsic.json'.format(abspath,file_name), metadata)

    # Open the file and load the JSON
    with open(abspath+"/" + file_name + "_intrinsic.json") as f:
        data = json.load(f)

    height = data['height']
    width = data['width']
    intrinsics = data["intrinsic_matrix"]
    time_stamp = data["stream_length_usec"]
    #print(f"Intrinsic Matrix {intrinsics}")

    camera_intrinsics = o3d.camera.PinholeCameraIntrinsic()
    cx = intrinsics[6]
    cy = intrinsics[7]
    fx = intrinsics[0]
    fy = intrinsics[4]
    camera_intrinsics.set_intrinsics(width,height,fx,fy,cx,cy)

    last_frame = None
    while not reader.is_eof(): # go until hitting eof because of exposure issues in early color frames
        rgbda = reader.next_frame();
        if rgbda is None:
            #print("Got nothing! ")
            continue
        last_frame = rgbda

    if last_frame is not None:
        #print("Got the last frame")
        rgbd_frames[i] = last_frame
    else:

        print("************No valid frames found in the .mkv file.**********")

    # filter the depth image for flying pixels
    # unfiltered PCD

    # Convert the RGBD data to tensors
    last_frame_t = o3d.t.geometry.Image.as_tensor(last_frame)
    
    #color_tensor = o3d.t.geometry.Image.from_legacy()   (last_frame.color, dtype=o3d.core.Dtype.Float32, device=o3d.core.Device("CUDA:0"))
    #depth_tensor = o3d.t.io.Image.to_tensor(last_frame.depth, dtype=o3d.core.Dtype.Float32, device=o3d.core.Device("CUDA:0"))

    # Construct the tensor-based RGBD image

In [ ]:
# Convert legacy images to numpy arrays
color_np = np.asarray(last_frame.color)
depth_np = np.asarray(last_frame.depth)

# Convert numpy arrays to tensors
color_tensor = o3d.core.Tensor(color_np, o3d.core.Dtype.Float32, o3d.core.Device("CPU:0"))
depth_tensor = o3d.core.Tensor(depth_np, o3d.core.Dtype.Float32, o3d.core.Device("CPU:0"))

# Create a tensor-based RGBDImage
rgbdt = o3d.t.geometry.RGBDImage(color_tensor, depth_tensor)
o3d.visualization.draw_geometries([rgbdt.to_legacy()])

In [ ]:
last_frame = None
while not reader.is_eof(): # go until hitting eof because of exposure issues in early color frames
    rgbda = reader.next_frame();
    if rgbda is None:
        #print("Got nothing! ")
        continue
    last_frame = rgbda

if last_frame is not None:
    #print("Got the last frame")
    rgbd_frames[i] = last_frame
else:

    print("************No valid frames found in the .mkv file.**********")
rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
last_frame.color, last_frame.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)

volume = o3d.pipelines.integration.ScalableTSDFVolume(
            voxel_length= 4.0/ 512.0,
            sdf_trunc=0.04,
            color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)

volume.integrate(
    rgbdc,
    camera_intrinsics,
    np.eye(4),
)


pcd_tsdf = volume.extract_point_cloud()
pcds_tsdf.append(pcd_tsdf)

In [ ]:
draw_registration_result(pt[0],pt[0],np.eye(4))

In [ ]:
x='/home/vigir3d/Datasets/cattle_scans/farm_scan1/Animal_482_2/'
cc=x.split("Animal")[-1:]
fname = cc.split
print("Animal"+cc[0])

In [ ]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])


def pick_points(pcd):
    print("")
    print(
        "1) Please pick at least three correspondences using [shift + left click]"
    )
    print("   Press [shift + right click] to undo point picking")
    print("2) After picking points, press 'Q' to close the window")
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    vis.run()  # user picks points
    vis.destroy_window()
    print("")
    return vis.get_picked_points()


def demo_manual_registration(src, tgt):
    print("Demo for manual ICP")
    source = src
    target = tgt
    print("Visualization of two point clouds before manual alignment")
    draw_registration_result(source, target, np.identity(4))

    # pick points from two point clouds and builds correspondences
    picked_id_source = pick_points(source)
    picked_id_target = pick_points(target)
    assert (len(picked_id_source) >= 3 and len(picked_id_target) >= 3)
    assert (len(picked_id_source) == len(picked_id_target))
    corr = np.zeros((len(picked_id_source), 2))
    corr[:, 0] = picked_id_source
    corr[:, 1] = picked_id_target

    # estimate rough transformation using correspondences
    print("Compute a rough transform using the correspondences given by user")
    p2p = o3d.pipelines.registration.TransformationEstimationPointToPoint()
    trans_init = p2p.compute_transformation(source, target,
                                            o3d.utility.Vector2iVector(corr))

    # point-to-point ICP for refinement
    print("Perform point-to-point ICP refinement")
    threshold = 0.03  # 3cm distance threshold
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    draw_registration_result(source, target, reg_p2p.transformation)
    return reg_p2p.transformation
    print("")


#if __name__ == "__main__":
    
    #t01 = demo_manual_registration(p[0],p[1])

In [ ]:
t = colored_ICP(pcds[0],pcds[1])

In [ ]:
t01 = register_two_views_teaser(pcds[0],pcds[1],0.05)
t12 = register_two_views_teaser(pcds[1],pcds[2],0.05)
t52 = register_two_views_teaser(pcds[5],pcds[2],0.05)
t43 = register_two_views_teaser(pcds[4],pcds[3],0.05)
t32 = register_two_views_teaser(pcds[3],pcds[2],0.05)

In [ ]:
draw_registratiaon_result(pcds[0],pcds[1],t)

In [ ]:
t01 = demo_manual_registration(pcds[0],pcds[1])

In [ ]:
t12 = demo_manual_registration(p[1],p[2])

In [ ]:
t52= demo_manual_registration(p[5],p[2])

In [ ]:
t43 = demo_manual_registration(p[4],p[3])

In [ ]:
t01 = colored_ICP(p[0],p[1])
draw_registration_result(p[0],p[1],t01)

In [ ]:
t01 = register_two_views_teaser(c[0],c[1],0.035)
t01 = register_two_views_teaser(c[1],c[2],0.045)
t01 = register_two_views_teaser(c[5],c[2],0.045)
t01 = register_two_views_teaser(c[4],c[3],0.05)
t01 = register_two_views_teaser(c[3],c[2],0.35)


In [ ]:
files_482 = glob.glob(data_path+"/*.ply")
files_482.sort()

pcds_482 = []
for file in files_482:
    pcds_482.append(o3d.io.read_point_cloud(file))

In [ ]:
r01,t01 = register_two_views_teaser(pcds_482[0],pcds_482[1],0.05)
r13,t13 = register_two_views_teaser(pcds_482[1],pcds_482[3],0.08)
#r56,t56 = register_two_views_teaser(pcds_482[5],pcds_482[6],0.05)
#r73,t73 = register_two_views_teaser(pcds_482[7],pcds_482[3],0.05)
#r67,t67 = register_two_views_teaser(pcds_482[6],pcds_482[7],0.08)

In [ ]:
H0 = t13 @ t01
H1 = t13
H5 = t73 @ t67 @ t56
H6 = t73 @ t67
H7 = t73

In [ ]:
p0 = copy.deepcopy(pcds_482[0]).transform(H0)
p1 = copy.deepcopy(pcds_482[1]).transform(H1)
p5 = copy.deepcopy(pcds_482[5]).transform(H5)
p6 = copy.deepcopy(pcds_482[6]).transform(H6)
p7 = copy.deepcopy(pcds_482[7]).transform(H7)

In [ ]:
o3d.visualization.draw_geometries([p0,p1,pcds_482[3],p5,p6,p7])

In [ ]:
r56,t56 = register_two_views_teaser(pcds_482[5],pcds_482[6],0.05)
r73,t73 = register_two_views_teaser(pcds_482[7],pcds_482[3],0.05)
r67,t67 = register_two_views_teaser(pcds_482[6],pcds_482[7],0.08)

In [ ]:
len(tforms_apriltag)

In [ ]:
len(pcds_482)

In [ ]:
c0 = copy.deepcopy(tforms_apriltag[0]).transform(H0)
c1 = copy.deepcopy(tforms_apriltag[1]).transform(H1)
c5 = copy.deepcopy(tforms_apriltag[3]).transform(H5)
c6 = copy.deepcopy(tforms_apriltag[4]).transform(H6)
c7 = copy.deepcopy(tforms_apriltag[5]).transform(H7)

o3d.visualization.draw_geometries([c0,c1,c5,c6,c7,tforms_apriltag[2]])

In [ ]:
for vs in np.arange(0.01,0.09,0.01):
    print(vs)
    a01,b01 = register_two_views_teaser(tforms_apriltag[0],tforms_apriltag[1],vs)
#r,p = register_two_views_teaser(tforms_apriltag[1],tforms_apriltag[2],0.05)
#r,p = register_two_views_teaser(tforms_apriltag[3],tforms_apriltag[2],0.05)
#r,p = register_two_views_teaser(tforms_apriltag[4],tforms_apriltag[3],0.05)
#r,p = register_two_views_teaser(tforms_apriltag[5],tforms_apriltag[2],0.05)

In [ ]:
o3d.visualization.draw_geometries([c0,c1,c5,c6,c7,tforms_apriltag[2]])

In [ ]:
import open3d as o3d
# Read the point cloud from a PLY file
point_cloud = o3d.io.read_point_cloud("/home/vigir3d/Desktop/animal_482_2_demo_cleaned_2.ply")

# Check the number of points in the point cloud
print(f"Point cloud has {len(point_cloud.points)} points.")

# Visualize the point cloud
o3d.visualization.draw_geometries([point_cloud])
